# Add group label for test dataset

In [ ]:
# import pandas as pd

# train_path = "data/Product10K/train.csv"
# test_path = "data/Product10K/test_kaggletest.csv"

# train_df = pd.read_csv(train_path)
# test_df = pd.read_csv(test_path)
# print(f"train: {train_df.shape}, test: {test_df.shape}")

In [ ]:
# test_df.head()

In [ ]:
# class_group = {}

# for i, row in train_df.iterrows():
#     if row['class'] not in class_group:
#         class_group[row['class']] = row['group']
#     else:
#         if class_group[row['class']] != row['group']:
#             print(f"mismatched class group")

# # class_group

In [ ]:
# test_df['group'] = test_df['class'].apply(lambda x: class_group[x])
# test_df.head()

In [ ]:
# test_df_group_label = pd.read_csv("data/Product10K/test_group_label.csv")
# test_df_group_label.head()

In [ ]:
# group_label = {}

# for i, row in test_df_group_label.iterrows():
#     if row['group'] not in group_label:
#         group_label[row['group']] = row['flag'].replace('/', '、').replace('-', '、').strip()

# print(len(group_label))
# group_label

In [ ]:
# import re

# group_label_path = "data/Product10K/group_label.csv"
# group_label_df = pd.DataFrame(list(group_label.items()), columns=['group', 'flag'])
# group_label_df = group_label_df.sort_values('group')

# group_label_df.to_csv(group_label_path, index=False)

In [ ]:
# group_label_path = "data/Product10K/group_label.csv"
# group_label_df = pd.read_csv(group_label_path)
# # group_label_df['label'] = group_label_df['label'].apply(lambda x: x.replace('-', '、'))
# # group_label_df.to_csv(group_label_path)
# group_label_df.head()

In [ ]:
# test_df = pd.merge(test_df, group_label_df, on='group')
# print(test_df.shape)
# test_df.head()

In [ ]:
# test_df = test_df.rename(columns={'name': 'filename', 'label': 'flag'})
# test_df.head()

In [ ]:
# test_df.to_csv("data/Product10K/test_wt_group_label.csv", index=False)

In [ ]:
# import boto3
# import json
# import base64
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import time
# import random

# # Initialize Bedrock client
# bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')

# def encode_image_to_base64(image_path):
#     """Convert image to base64 string"""
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# def call_claude_for_group(group_num, image_names):
#     """Call Claude 3.7 to get group label for a set of images"""
#     try:
#         # Prepare images for Claude
#         images_data = []
#         for img_name in image_names[:5]:  # Take only first 5 images
#             img_path = os.path.join("data/Product10K/test", img_name)
#             if os.path.exists(img_path):
#                 img_base64 = encode_image_to_base64(img_path)
#                 images_data.append({
#                     "type": "image",
#                     "source": {
#                         "type": "base64",
#                         "media_type": "image/jpeg",
#                         "data": img_base64
#                     }
#                 })
        
#         if not images_data:
#             return group_num, "No valid images found"
        
#         # Prepare the message content
#         content = [
#             {
#                 "type": "text",
#                 "text": "请看这些产品图片，提供一个简洁准确的细分商品类别标签，最好地描述这些是什么类型的产品。专注于主要的产品类别（例如：'电子产品'、'服装'、'家居用品'、'运动器材'等）。请只回答类别名称，不需要额外说明。"
#             }
#         ] + images_data
        
#         # Prepare the request
#         request_body = {
#             "anthropic_version": "bedrock-2023-05-31",
#             "max_tokens": 50,
#             "temperature": 0.1,
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": content
#                 }
#             ]
#         }
        
#         # Call Bedrock
#         response = bedrock_runtime.invoke_model(
#             modelId="anthropic.claude-3-5-sonnet-20241022-v2:0",
#             body=json.dumps(request_body)
#         )
        
#         # Parse response
#         response_body = json.loads(response['body'].read())
#         label = response_body['content'][0]['text'].strip()
        
#         print(f"Group {group_num}: {label}")
#         return group_num, label
        
#     except Exception as e:
#         print(f"Error processing group {group_num}: {str(e)}")
#         return group_num, f"Error: {str(e)}"

# # Get unique groups and sample images
# unique_groups = test_df['group'].unique()
# print(f"Total groups to process: {len(unique_groups)}")

# # Prepare data for concurrent processing
# group_data = []
# for group in unique_groups:
#     group_images = test_df[test_df['group'] == group]['name'].tolist()
#     # Randomly sample 5 images from the group
#     sampled_images = random.sample(group_images, min(5, len(group_images)))
#     group_data.append((group, sampled_images))

# # Process groups concurrently
# group_labels = {}
# max_workers = 5  # Limit concurrent requests to avoid rate limits

# print("Starting concurrent Claude API calls...")
# with ThreadPoolExecutor(max_workers=max_workers) as executor:
#     # Submit all tasks
#     future_to_group = {
#         executor.submit(call_claude_for_group, group, images): group 
#         for group, images in group_data
#     }
    
#     # Process completed tasks
#     for future in as_completed(future_to_group):
#         group_num, label = future.result()
#         group_labels[group_num] = label
        
#         # Add small delay to respect rate limits
#         time.sleep(0.1)

# print(f"\nCompleted processing {len(group_labels)} groups")
# print("\nGroup Labels:")
# for group, label in sorted(group_labels.items()):
#     print(f"Group {group}: {label}")


In [ ]:
# group_labels[5] = "服装 - 男装冬季服饰"
# group_labels[52] = "日用百货"
# group_labels[55] = "皮具配饰/钱包"
# group_labels[73] = "文具用品/钢笔"
# group_labels[100] = "打印机耗材"
# group_labels[136] = "手机配件"
# group_labels[187] = "饮品和食品类/早餐谷物"
# group_labels[194] = "玩具模型/玩具类"
# group_labels[197] = "儿童玩具/益智玩具"
# group_labels[201] = "工艺品/装饰品"
# group_labels[204] = "服饰配件/皮带"
# group_labels[210] = "服饰配件"
# group_labels[213] = "服饰配件/皮带"
# group_labels[215] = "儿童用品"
# group_labels[238] = "个人护理用品"
# group_labels[305] = "护理保健品"
# group_labels[329] = "家居用品/装饰品"
# group_labels[348] = "家居装饰品"
# group_labels[351] = "家居用品/晾衣架"
# group_labels[358] = "家居装饰品/花瓶"

In [ ]:
# import json
# import re

# group_label_path = "data/Product10K/group_label.csv"

# group_label_df = pd.DataFrame(list(group_labels.items()), columns=['group', 'label'])
# group_label_df = group_label_df.sort_values('group')
# for i, row in group_label_df.iterrows():
#     label = row['label']
#     label = re.sub(r' - ', '/', label)
#     group_label_df.loc[i, 'label'] = label.strip()

# group_label_df.to_csv(group_label_path, index=False)
# group_label_df.head()


In [ ]:
# len(group_label_df['label'].unique())

In [ ]:
# df_merged = pd.merge(test_df, group_label_df, on='group')
# df_merged.head()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

# Specify group number
group_number = 50

# Filter images from the specified group
group_images = test_df[test_df['group'] == group_number]['name'].head(10).tolist()

# Create figure with 2 rows, 5 columns
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle(f'Group {group_number} Images', fontsize=16)

# Display images
for i, img_name in enumerate(group_images):
    row = i // 5
    col = i % 5
    
    # Load and display image
    img_path = os.path.join("data/Product10K/test", img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[row, col].imshow(img)
        axes[row, col].set_title(img_name, fontsize=8)
        axes[row, col].axis('off')
    else:
        axes[row, col].text(0.5, 0.5, f'{img_name}\nNot Found', 
                           ha='center', va='center', transform=axes[row, col].transAxes)
        axes[row, col].axis('off')

# Hide empty subplots if less than 10 images
for i in range(len(group_images), 10):
    row = i // 5
    col = i % 5
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print(f"Showing {len(group_images)} images from group {group_number}, {group_label_df[group_label_df['group']==group_number]}")


# Prepare Nova SFT data

## Shein data set

In [ ]:
# import pandas as pd
# import json

# # Constants
# account_id = "452145973879"
# s3_bucket = "sagemaker-us-east-1-452145973879"
# # s3_prefix = "projects/nova-image-tagging/data"

# system_prompt = """# Image Classification Assistant
    
#     ## Role
#     You are an image analyzer specialized in classifying visual content according to provided reference categories.
    
#     ## Analysis Framework
#     You must follow this 4-step process for every image:
    
#     ### Step 1: Physical Attribute Extraction
#     - Extract the objective physical attributes of all items in the image.
#     - Focus strictly on neutral, geometric, and material properties.
#     - **DO NOT** interpret the item's purpose, function, or potential use.
#     - **Examples of attributes to extract:**
#     -   **Shape:** Cylindrical, rectangular, pointed tip, curved edge, ring-shaped.
#     -   **Components:** Handle, blade, trigger, nozzle, switch, USB port, chain.
#     -   **Material/Texture:** Metallic, plastic, fabric, wooden, smooth, serrated.
#     -   **Text/Markings:** Any visible letters, numbers, or symbols.
    
#     ### Step 2: Feature Analysis
#     Based on your description, identify key features:
#     - Look for specific shapes (cylindrical, pointed, etc.)
#     - Identify materials (metal, plastic, fabric, etc.)
#     - Note functional elements (buttons, switches, blades, etc.)
#     - Check for size indicators or child-oriented designs
    
#     ### Step 3: Category Matching
#     Compare identified features with the reference categories:
#     - Match observed features with the "Key Features" listed for each category
#     - Consider the "Definition" to understand the intended use or design purpose
#     - Determine if the item meets the criteria for any category
    
#     ### Step 4: Final Classification
#     Provide classification result following these rules:
#     - Return only the Chinese category names (the values in parentheses)
#     - If multiple categories apply, separate them with English commas ","
#     - Maximum of 5 categories
#     - If no categories match, return "无"
#     - Format your response as JSON: {"result":"target"}
    
#     ## Output Requirements
#     - Perform Steps 1-3 internally (do not show your analysis)
#     - Only output the final JSON result from Step 4
#     - Use Chinese category names only
#     - Follow the exact JSON format specified
    
#     ## Important Guidelines
#     - Be precise - only classify items that clearly match the definitions
#     - Consider both definition and key features when classifying
#     - If uncertain about a match, err on the side of caution and exclude it
#     - Do NOT include escaped Unicode characters in your output

#     ## Reference Categories

#     ### Knives and Blades
#     1. **匕首 (Dagger)**  
#        - **Definition**: A short-bladed weapon designed primarily for thrusting or stabbing attacks, characterized by a sharp pointed tip and compact design optimized for single-hand use.
#        - **Key Features**: Sharp pointed tip (essential), short blade, designed for thrusting, single or double-edged, compact for close-quarters use.
    
#     2. **刀具 (Knives)**  
#        - **Definition**: Includes all types of knives, such as utility, tactical, folding, or ornamental blades, designed for cutting or slashing.  
#        - **Key Features**: Sharp blade edges, handle designs (with or without finger rings), sheath or folding mechanisms, blade-like structures.

#     3. **弹簧刀 (Switchblade)**  
#        - **Definition**: A folding knife with a blade that deploys automatically via a spring mechanism, often triggered by a button.  
#        - **Key Features**: Button or lever on handle, blade concealed in handle when folded, rapid deployment action implied by design.
    
#     4. **蝴蝶刀 (Butterfly Knife)**  
#        - **Definition**: A folding pocket knife with two handles that counter-rotate around the tang, allowing the blade to be concealed or deployed.  
#        - **Key Features**: No curved handle, dual pivoting handles, blade sandwiched between handles when closed, often metallic with latch mechanism.
    
#     5. **伪装刀具 (Disguised Knife)**  
#        - **Definition**: Knives concealed within everyday objects, such as pens, credit cards, or belts, to evade detection.  
#        - **Key Features**: Hidden blade mechanism, innocuous outer appearance (e.g., lipstick case, keychain), deployable sharp edge.
    
#     6. **爪刀 (Claw Knife)**  
#        - **Definition**: A knife designed for slashing or gripping, featuring a curved or slightly straight blade with one or more finger rings or holes in the handle for enhanced grip and hand-worn capability. Includes tactical, folding, and ornamental variations, distinguished by the presence of finger rings.  
#        - **Key Features**: One or more finger holes or rings in the handle as the primary identifier, curved or slightly straight blade.
    
#     ### Weapons and Restricted Items
    
#     7. **仿真子弹 (Imitation Bullet)**  
#        - **Definition**: Replica ammunition, often used in keychains or necklaces (e.g., bullet keychains, bullet necklaces).  
#        - **Key Features**: Cylindrical casing with pointed tip, metallic sheen, no live components.
    
#     8. **警用喷雾 (Police Spray)**  
#        - **Definition**: Aerosol sprays including pepper, chili, tear gas, or anti-wolf/safety sprays.  
#        - **Key Features**: Canister with nozzle, warning labels, handheld design.
    
#     9. **警棍/甩棍 (Baton/Swing Stick)**  
#        - **Definition**: Prohibited metal batons made of high-strength alloy steel, often with aggressive protrusions, some with police markings.  
#        - **Key Features**: Telescoping sections, metallic finish, protrusions (e.g., spikes, barbs), police insignia.
    
#     10. **电击器/电击棒 (Stun Gun/Stun Baton)**  
#         - **Definition**: Devices with exposed high-voltage electrodes and aggressive protrusions, producing electric arcs.  
#         - **Key Features**: Exposed electrodes, protrusions (e.g., spikes, barbs), visible electric arc or light.
    
#     11. **指虎铁莲花 (Knuckle Duster/Iron Lotus)**  
#         - **Definition**: Hand-worn offensive weapons including knuckle dusters, push swords, wolverine claws, iron lotus, finger rings, or key sticks.  
#         - **Key Features**: Metal rings for fingers, protruding spikes or blades, fist-enclosing design.
    
#     ### Explosive or Hazardous Items
    
#     12. **卡式炉 (Cassette Stove)**  
#         - **Definition**: A portable cooking stove using gas canisters.  
#         - **Key Features**: Burner head, gas canister slot, foldable legs, ignition switch.
    
#     13. **烟花爆竹 (Fireworks and Firecrackers)**  
#         - **Definition**: A pyrotechnic product that produces bright sparks by igniting a chemical coating.
#         - **Key Features**: Rod/cylinder; produces sparks and flame; requires ignition; often has a pointed base for insertion; typically covered in glitter/metallic finish.
    
#     14. **射钉弹 (Nail Gun Cartridge)**  
#         - **Definition**: Ammunition for nail guns, containing explosive charges to drive nails.  
#         - **Key Features**: Cylindrical cartridge with metallic tip, labeled as ammunition, compatible with nail gun devices.
    
#     15. **打火石、打火棒 (Flint or Fire Starter)**  
#         - **Definition**: Tools using flint or metal rods to generate sparks for fire starting.  
#         - **Key Features**: Metallic rod or stone, striker tool, spark-generating surface.
    
#     16. **气罐 (Gas Canister)**  
#         - **Definition**: Pressurized containers banned for cross-border transport, containing flammable or compressed gases.  
#         - **Key Features**: Cylindrical metal container, pressure valve, hazard labels, sealed cap.
    
#     17. **压罐喷雾 (Aerosol Spray Can)**  
#         - **Definition**: Pressurized cans dispensing flammable or hazardous sprays.  
#         - **Key Features**: Metal canister, nozzle with trigger, pressure release valve, warning labels.
    
#     18. **烟雾弹 (Smoke Bomb)**  
#         - **Definition**: Devices releasing colored smoke, often used for signaling or entertainment.  
#         - **Key Features**: Small cylindrical or spherical shape, fuse or ignition point, smoke-emitting vent.
    
#     19. **固体酒精 (Solid Alcohol)**  
#         - **Definition**: Solid fuel blocks used for heating or cooking.  
#         - **Key Features**: Rectangular or disc-shaped blocks, wax or gel-like texture, flammable packaging.
    
#     20. **镁粉 (Magnesium Powder)**  
#         - **Definition**: Fine magnesium powder used as a flammable material.  
#         - **Key Features**: Silvery powder, airtight packaging, flammable warning labels.
    
#     21. **烟饼（片） (Smoke Cake/Piece)**  
#         - **Definition**: Compact smoke-producing devices, often for rituals or effects.  
#         - **Key Features**: Flat or disc shape, ignition point, smoke-emitting surface.
    
#     22. **重燃蜡烛 (Relight Candles)**  
#         - **Definition**: Candles designed to relight after being blown out.  
#         - **Key Features**: Waxed wick with embedded relighting mechanism, candle shape, flammable material.
    
#     23. **火柴 (Matches)**  
#         - **Definition**: Small sticks with a combustible head for ignition.  
#         - **Key Features**: Wooden or paper sticks, colored ignitable tip, striking surface.
    
#     24. **炸药 (Explosives)**  
#         - **Definition**: High-explosive materials used for demolition or blasting.  
#         - **Key Features**: Block or granular form, industrial packaging, explosive hazard symbols, detonation wiring.
    
#     25. **红磷、白磷 (Red Phosphorus/White Phosphorus)**  
#         - **Definition**: Highly reactive phosphorus compounds used in explosives or incendiaries.  
#         - **Key Features**: Red or white crystalline powder, sealed containers, chemical hazard labels.
    
#     26. **火药 (Gunpowder)**  
#         - **Definition**: A combustible powder used as a propellant or explosive.  
#         - **Key Features**: Granular or fine powder appearance, packaging with explosive warnings, metallic or plastic containers.
    
#     27. **礼花筒 (Firework Tube)**  
#         - **Definition**: A celebratory device that ejects confetti or streamers through mechanical action (non-combustible).
#         - **Key Features**: Cylindrical tube; ejects paper/streamers; no fire or heat; often has twisting/pushing mechanism at base.
    
#     ### Electronics and Appliances
    
#     28. **充电宝 (Power Bank)**  
#         - **Definition**: A portable device used to charge other electronic devices via a built-in battery.  
#         - **Key Features**: Rectangular or cylindrical shape, USB ports (input/output), battery indicator lights, compact design.
    
#     29. **纯锂电池 (Pure Lithium Battery)**  
#         - **Definition**: Standalone lithium batteries, identifiable by CR/BR markings or labeled as "lithium battery/锂电池," excluding those sold with other products.  
#         - **Key Features**: CR/BR prefix on labeling, cylindrical or coin-shaped, metallic casing, no accompanying devices.
    
#     30. **电子打火器 (Electronic Lighter)**  
#         - **Definition**: Lighters using piezoelectric crystals to generate high-voltage sparks, including rechargeable or battery-powered models.  
#         - **Key Features**: Pressable crystal or button, electrode gap for sparks, charging port or battery compartment.
    
#     31. **打火器（无燃气） (Gas-Free Lighter)**  
#         - **Definition**: Lighters or electronic ignition devices without gas fuel, including flint or piezoelectric types.  
#         - **Key Features**: Flint wheel or piezoelectric crystal, no visible gas tank, compact ignition design.
    
#     32. **燃气打火机 (Gas Lighter)**  
#         - **Definition**: Lighters powered by flammable gas (e.g., butane).  
#         - **Key Features**: Gas tank visible, ignition trigger, flame adjustment knob, compact design.

#     ### Children's Products
#     33. **儿童包 (Children's Bag)**  
#         - **Definition**: A bag designed specifically for children, varying in size and purpose, used for carrying personal items, school supplies, or toys.  
#         - **Key Features**: Playful designs, cartoon characters, or themed motifs. Secure zipper closure. Various carrying options (wrist, handle, shoulder). Child-sized, with compartments or single storage, sometimes with charms.

#     34. **童鞋 (Children's Shoes)**  
#         - **Definition**: Shoes for children, indicated by text, images, or sizes (foot length ≤240mm).  
#         - **Key Features**: Small size, child-friendly design, labeled size range.

#     35. **婴童帽子 (Infant/Toddler Hat)**  
#         - **Definition**: Headwear specifically designed for infants and toddlers, primarily for warmth or sun protection.  
#         - **Key Features**: Soft, comfortable material, providing warmth or sun protection. Often features cartoon patterns, animal shapes, or cute prints. May include ear flaps, chin straps, or pom-pom decorations for comfort and secure fit.

#     36. **儿童发饰 (Children's Hair Accessories)**  
#         - **Definition**: Decorative hair clips, headbands, or hair ties specifically designed for children, used to secure or adorn hairstyles.  
#         - **Key Features**: Brightly colored and varied in shape (e.g., bows, stars, butterflies), often featuring cartoon patterns or cute embellishments. Typically secured with clips, elastic bands, or ties, ensuring they are safe and easy to wear.

#     37. **儿童太阳镜 (Children's Sunglasses)**  
#         - **Definition**: Eyewear specifically designed for children to protect their eyes from the sun's harmful UV rays, often featuring playful designs.  
#         - **Key Features**: UV protection, durable and child-friendly materials, various fun shapes (e.g., heart, flower, classic, cat-eye), colorful frames and lenses, and sometimes decorative elements like glitter or animal ears.

#     38. **儿童自行车座椅 (Child Bicycle Seat)**  
#         - **Definition**: A bicycle seat designed for children, indicated by text or images claiming child suitability.  
#         - **Key Features**: Small seat attached to bike frame, child-sized design, safety straps or padding.

#     ### Other Restricted Items
#     39. **手铐、拇指烤、脚镣 (Handcuffs/Thumb Cuffs/Leg Irons)**  
#         - **Definition**: Metal restraints meeting size criteria: handcuffs (>5cm inner diameter), thumb cuffs (>1cm), leg irons (>8cm).  
#         - **Key Features**: Metal construction, hinged or linked design, size consistent with restraint purpose."""

# # system_prompt = "请分析图片。"
# user_prompt = "Please classify the item in this image according to the categories defined in the system."

# def create_record(flag, s3_prefix, filename):
#     s3_path = f"s3://{s3_bucket}/{s3_prefix}/{filename}"

#     img_format = "jpeg" if filename.endswith(".jpg") else filename.split('.')[-1]

#     if img_format not in ['jpeg']:
#         # print(img_format)
#         return None
    
#     return {
#         "schemaVersion": "bedrock-conversation-2024",
#         "system": [{
#             "text": system_prompt
#         }],
#         "messages": [{
#                 "role": "user",
#                 "content": [
#                     {
#                         "image": {
#                             "format": img_format,
#                             "source": {
#                                 "s3Location": {
#                                     "uri": s3_path,
#                                     "bucketOwner": account_id
#                                 }
#                             }
#                         }
#                     },
#                     {	"text": user_prompt	}
#                 ]
#             },
#             {
#                 "role": "assistant",
#                 "content": [{
#                     "text": flag
#                 }]
#             }
#         ]
#     }

In [ ]:
import pandas as pd
import json

# Constants
account_id = "452145973879"
s3_bucket = "sagemaker-us-east-1-452145973879"
# s3_prefix = "shein_img_tagging/imgs"
system_prompt = """##ROLE##
You are an advanced image classification specialist analyzing e-commerce product images to identify multiple relevant category tags from the ##REFERENCE_CATEGORIES##. Your primary focus is comprehensive coverage (recall) over strict precision.

##PRIMARY_TASK##
Analyze the provided image and identify **UP TO 5 MOST RELEVANT** category tags from ##REFERENCE_CATEGORIES##. You are **STRONGLY ENCOURAGED** to output multiple tags when the image could reasonably belong to several categories or when there's reasonable uncertainty.

##CRITICAL_MULTI_LABEL_REQUIREMENTS##

**IMPORTANT: Multiple tags Are Expected and Preferred**
- **Output 1-3 tags** whenever multiple categories from ##REFERENCE_CATEGORIES## are plausible
- Think of this as providing comprehensive classification alternatives, not just the single best match
- Consider related categories, different levels of specificity, and various product aspects
- **Even if one label seems most accurate, include other reasonable alternatives**
- **Err on the side of inclusion** - it's better to flag potential matches than miss restricted items

**When to Output Multiple tags (ENCOURAGED)**
- The product could belong to multiple related categories from ##REFERENCE_CATEGORIES##
- Different components or features suggest different classifications  
- Similar items with slight variations might fit different categories
- There's reasonable uncertainty between closely related categories
- Borderline cases that might pose compliance risks
- Items that share characteristics with multiple restricted categories

##ANALYSIS_FRAMEWORK##
Follow this enhanced 4-step process using chain-of-thought reasoning:

**Step 1: Comprehensive Physical Attribute Extraction**
- Extract **ALL** objective physical attributes of **ALL** items in the image
- Focus on neutral, geometric, and material properties
- **DO NOT** interpret purpose or function yet - just observe
- **Examples of attributes to extract:**
  - **Shape:** Cylindrical, rectangular, pointed tip, curved edge, ring-shaped, telescoping
  - **Components:** Handle, blade, trigger, nozzle, switch, USB port, chain, electrodes
  - **Material/Texture:** Metallic, plastic, fabric, wooden, smooth, serrated, crystalline
  - **Text/Markings:** Any visible letters, numbers, symbols, warnings, brand names
  - **Size indicators:** Dimensions, child-oriented designs, scale references

**Step 2: Multi-Aspect Feature Analysis**
Based on your description, identify **ALL** potentially relevant features:
- **Primary features:** Main identifying characteristics
- **Secondary features:** Additional elements that might suggest other categories
- **Contextual clues:** Usage scenarios, accompanying items, packaging
- **Safety indicators:** Warning labels, hazard symbols, age restrictions
- **Design intent:** Child-friendly designs, tactical appearances, concealment features

**Step 3: Comprehensive Category Matching**
Compare identified features with **ALL** categories in ##REFERENCE_CATEGORIES##:
- **Match observed features** with the "Key Features" listed for each category
- **Consider multiple perspectives** - how different reviewers might categorize this item
- **Include borderline cases** that might pose compliance risks
- **Don't dismiss close matches** - include categories that are "reasonably close"
- **Think about related categories** that share similar characteristics
- **Consider both obvious and subtle matches**

**Step 4: Multi-Label Classification Output**
Provide classification following these **enhanced rules**:
- **PREFERRED: Output 2-5 category names** when multiple reasonable matches exist
- **Include borderline cases** that human reviewers should evaluate
- **Order by confidence level** (highest confidence first)
- **Use ONLY the OUTPUT_LABEL values** from ##REFERENCE_CATEGORIES## for all matches
- Separate multiple categories with English commas "," only
- **Single label output should be exceptional** - only when truly only one category applies
- If absolutely no categories match, return "无" (this should be rare)
- Format: {"result":"category1,category2,category3"}

##OUTPUT_REQUIREMENTS##
- **Strongly favor multi-label outputs** - comprehensive coverage is key
- **Include "reasonable doubt" cases** - let human reviewers make final decisions  
- **Think broadly** about the image and provide multiple classification perspectives
- **Use ONLY the OUTPUT_LABEL values from ##REFERENCE_CATEGORIES##** for classification results
- Follow exact JSON format: {"result":"category1,category2"}
- **When uncertain between categories, include both rather than choosing one**
- **DO NOT include escaped Unicode characters** in output

##COMPLIANCE_GUIDELINES##
**Critical Guidelines for E-commerce Compliance:**
- **Prioritize recall over precision** - better to flag potential issues than miss them
- **Include items that could reasonably match** definitions from ##REFERENCE_CATEGORIES##, even with uncertainty
- **Borderline cases should be included** rather than excluded
- **Multiple perspectives benefit compliance** - provide comprehensive options
- **Focus on identifying potential compliance risks** across all applicable categories
- **Consider how different reviewers might interpret** the same image
- **Remember: Your role is to catch potential issues, not make final judgments**

##EXECUTION_INSTRUCTIONS##
You must strictly follow ##ANALYSIS_FRAMEWORK## and ##OUTPUT_REQUIREMENTS##. 
- Follow the 4-step analysis framework using chain-of-thought reasoning, then provide your response directly in the required JSON format without showing your analysis steps
- Refer to ##REFERENCE_CATEGORIES## for all classification decisions
- Apply ##COMPLIANCE_GUIDELINES## throughout your analysis

## Reference Categories
{"tag": "方向盘手机支架", "description": "安装于方向盘，用于固定手机。因分散驾驶员注意力、阻碍安全气囊，存在严重安全隐患。"}
{"tag": "放射性设备", "description": "包含放射性物质或能发射紫外线、激光、微波等能量的设备，可能对人体造成潜在伤害。"}
{"tag": "成人带电服装", "description": "集成电池、电路等电子元件的成人服饰，可实现发光、加热或震动等特殊功能，存在电击风险。"}
{"tag": "磁吸鼻钉/舌钉", "description": "无需穿孔，利用磁铁吸附在鼻、舌等部位的饰品。磁力强劲，存在被儿童误吞或吸入的风险。"}
{"tag": "铁粉加热鞋垫", "description": "内置铁粉，通过与空气接触发生氧化反应发热的鞋垫。提供一次性足部保暖，需防止低温烫伤。"}
{"tag": "有刺伤,划伤隐患的服装", "description": "带有锋利金属铆钉、尖锐装饰的服装，在穿着或接触时易造成使用者皮肤被刺伤、划伤。"}
{"tag": "其他安全隐患类服装配饰", "description": "包括易缠绕的长款饰品、含小零件易脱落的配饰，对特定人群（如儿童）构成潜在危险。"}
{"tag": "含烃类钥匙扣", "description": "内部含有流动液体或凝胶的钥匙扣。若外壳破损，液体泄漏后可能被吸入或误食，对健康有害。"}
{"tag": "可移动酒精壁炉", "description": "桌面级或小型的便携式取暖装饰品，使用酒精等液体燃料。因易倾倒，存在极高的火灾隐患。"}
{"tag": "孔明灯", "description": "由纸质灯罩和底部固体燃料构成。依靠明火升空，飞行不可控，极易引发火灾，属违禁品。"}
{"tag": "灭火器", "description": "内含干粉或二氧化碳等灭火剂的便携式压力容器。用于扑救初期火灾，运输和存储有特殊要求。"}
{"tag": "汽车安全带插扣/脱扣器", "description": "可插入安全带卡扣，用于消除未系安全带提示音的装置。它会误导车辆安全系统，极度危险。"}
{"tag": "汽车方向盘贴", "description": "粘贴或覆盖于方向盘中央或握把处的装饰品。可能影响安全气囊正常弹出或导致驾驶时打滑。"}
{"tag": "手握除草机&配件", "description": "手持式园林工具，配备高速旋转的金属刀片或链条。因其切割部件外露，存在高伤害风险。"}
{"tag": "汽车车牌遮挡", "description": "可通过遥控或手动操作，翻转或用帘布遮挡号牌的装置，用于非法逃避交通监控。"}
{"tag": "专业登山扣", "description": "用于攀岩、救援等生命安全相关活动的高强度锁扣。其性能需通过专业认证，以防发生坠落。"}
{"tag": "插入式导管", "description": "用于尿道刺激的成人性玩具，通常为细长棒状。因其用法特殊，对卫生和材质要求极高。"}
{"tag": "壮阳增大产品", "description": "宣称能增强男性功能或增大尺寸的外用油、膏、凝胶类产品。其功效和成分通常未经科学验证。"}
{"tag": "宠物染发剂", "description": "专用于改变宠物毛发颜色的化学染剂。可能含有刺激性成分，对宠物皮肤和健康存在潜在风险。"}
{"tag": "其他宠物食品（除猫/狗/马/鱼）", "description": "除猫、狗、马、鱼之外的其他宠物（如仓鼠、兔子、鸟类）的专用食品，需符合检疫标准。"}
{"tag": "一般宠物食品（猫狗马鱼）", "description": "专为猫、狗、马、鱼等常见宠物设计的主粮、零食或罐头。跨境销售需遵守进口国检疫法规。"}
{"tag": "宠物药品", "description": "用于预防或治疗宠物疾病的专用药物，如驱虫药、抗生素等。需在兽医指导下使用以确保安全。"}
{"tag": "电视机顶盒", "description": "接收卫星、有线或网络信号，并将其转换为电视可播放内容的电子设备，用于扩展电视频道。"}
{"tag": "无漏保吹风机/热风梳", "description": "指插头处未集成漏电保护开关的电吹风。在潮湿环境（如浴室）使用时，有极高的触电风险。"}
{"tag": "动物捕杀工具", "description": "用于捕获或杀伤动物的装置，如捕兽夹、索套陷阱等。其设计具有攻击性，对人畜均有威胁。"}
{"tag": "毒品", "description": "指鸦片、海洛因、冰毒、大麻等受国家法律管制的麻醉药品和精神药品，滥用会成瘾并违法。"}
{"tag": "吸毒工具", "description": "用于吸食、注射或加工毒品的工具，如冰壶、锡纸、注射器等。持有和使用此类物品涉嫌违法。"}
{"tag": "易制毒成分", "description": "指麻黄碱、丙酮等可用于非法制造毒品的化学原料。这些化学品受到严格管制，禁止随意买卖。"}
{"tag": "制毒工具-胶囊填充机", "description": "用于将粉末填充进空心胶囊的设备。因其可能被用于非法分装毒品而受到严格管控。"}
{"tag": "赌博用筹码", "description": "在赌场等赌博活动中，用作代币进行下注的圆形或方形牌。通常标有不同面值，代表货币。"}
{"tag": "针管", "description": "由针筒和活塞组成的医疗器械，用于注射液体或抽血。因可能被用于注射毒品而受销售管制。"}
{"tag": "止鼾贴（贴在嘴巴上)/呼吸贴", "description": "一种贴于唇部的胶带，用于在睡眠时物理闭合嘴巴，强制用鼻呼吸以减少打鼾。存在窒息风险。"}
{"tag": "匕首", "description": "双刃或单刃的短款刺击兵器，刀身对称，设计用于穿刺。属于管制刀具，具有高危险性。"}
{"tag": "超长刀具", "description": "刀刃长度超过15厘米的刀具。因其尺寸和潜在杀伤力，通常被归类为管制刀具，受法律限制。"}
{"tag": "弹簧刀", "description": "刀身可借助弹簧力量自动弹出的折叠刀。因其能快速展开，被视为攻击性武器，属于管制刀具。"}
{"tag": "蝴蝶刀", "description": "由两片可旋转的手柄包覆刀身的折叠刀，可通过甩动快速开合。属于管制刀具，具有危险性。"}
{"tag": "灰鲭鲨刀", "description": "一种设计独特的战术格斗刀，刀型具有强烈的攻击性。因其设计用途，被视为管制武器。"}
{"tag": "伪装刀具", "description": "外观伪装成梳子、口红、钥匙等日常用品的刀具。具有隐蔽性和突然性，危险程度高。"}
{"tag": "爪刀", "description": "一种模仿虎爪的弧形短刀，握柄处通常带有指环，便于反握和旋转。属于格斗用管制刀具。"}
{"tag": "弹弓", "description": "利用弹力皮筋发射弹丸的Y形手持投射工具。可发射钢珠等硬物，具有一定杀伤力。"}
{"tag": "弹弓弩", "description": "结合了弹弓的弹力发射和弩的扳机结构，威力更大，属于严格管制的武器。"}
{"tag": "飞镖", "description": "前端带有尖锐金属头，用于投掷的运动或武器。专业级或带刃飞镖具有杀伤力，受到管制。"}
{"tag": "弓", "description": "利用弓臂弹力发射箭矢的远射武器。竞技弓和狩猎弓因其威力，在运输和销售上受到限制。"}
{"tag": "箭/箭头", "description": "配合弓或弩使用的投射物。特别是带有锋利金属头的狩猎箭头，被视为武器配件，受到管制。"}
{"tag": "弩", "description": "带有扳机和箭槽的机械弓，发射威力远大于普通弓。因其强大杀伤力，被列为严格管制的武器。"}
{"tag": "双截棍", "description": "由两条短棍通过链条或绳索连接而成的武术器械。使用时具有较大冲击力，被视为攻击性武器。"}
{"tag": "伪装牙签", "description": "外观看似普通牙签或牙签盒，但实际隐藏着锋利尖刺的武器，具有极高的隐蔽性和危险性。"}
{"tag": "钥匙棍", "description": "外形类似钥匙的便携式防身武器，通常为金属材质，末端尖锐，可用于击打或穿刺。"}
{"tag": "仿真枪", "description": "外形、颜色和尺寸与真枪相似的仿制品。易引起恐慌或被用于犯罪活动，受到严格法律管制。"}
{"tag": "仿真子弹", "description": "外形模仿真实子弹的装饰品，如项链、钥匙扣等。虽无功能，但易造成误解，属违禁品类。"}
{"tag": "弹匣", "description": "为枪支供应弹药的可拆卸装置。是枪支的核心配件，属于严格管制的军用物品，严禁销售。"}
{"tag": "弹匣装弹器", "description": "用于快速向弹匣内压装子弹的辅助工具。作为枪支功能性配件，属于违禁品，禁止销售。"}
{"tag": "瞄准镜", "description": "安装于枪支上，用于精确瞄准目标的光学仪器。作为枪支关键配件，属于严格管制的物品。"}
{"tag": "枪管", "description": "枪支中引导弹头发射的管状部件。是枪支最核心的部件之一，属于绝对违禁品，严禁销售。"}
{"tag": "枪托", "description": "安装在枪身尾部，用于抵肩射击的部件。作为枪支的主要组成部分，属于违禁品，禁止销售。"}
{"tag": "枪支导轨", "description": "安装在枪身上的标准化接口，用于加装瞄准镜、手电等附件。属于枪支专用配件，禁止销售。"}
{"tag": "枪支握把", "description": "枪支上供手部握持的部件，用于提升操控稳定性。作为枪支专用配件，属于违禁品，禁止销售。"}
{"tag": "消音器", "description": "安装在枪口，用于降低射击时噪音和火焰的装置。是军用管制配件，严禁在民用市场销售。"}
{"tag": "粉刺针", "description": "一端或两端带有尖头、圆环的金属工具，用于物理方式清理粉刺。使用不当易造成皮肤感染。"}
{"tag": "听诊器", "description": "用于听诊人体心、肺等器官声音的医疗诊断工具。通常由拾音头、导管和耳塞组成。"}
{"tag": "微针滚轮", "description": "布满微小针头的滚轮，在皮肤上滚动制造微创，以刺激胶原蛋白再生。属医疗美容器械。"}
{"tag": "土壤", "description": "天然土壤或盆栽用土。因可能携带病虫害或外来物种，跨国邮寄受到严格的检疫法规限制。"}
{"tag": "玩具无人机", "description": "通常尺寸较小、功能简单，主要供儿童娱乐的无人机。虽为玩具，仍需在安全区域飞行。"}
{"tag": "消费级无人机", "description": "配备高清摄像头和GPS等功能，用于航拍和娱乐的无人机。飞行受各国空域法规的严格管制。"}
{"tag": "夜视望远镜", "description": "利用微光或红外技术，在夜间或黑暗环境中观察目标的望远镜。部分高性能产品可能受出口管制。"}
{"tag": "干植物", "description": "经过干燥处理的植物全体或部分，如干花、干草等。因可能携带病虫，跨境邮寄受检疫限制。"}
{"tag": "植物活体", "description": "具有生命活性的植物，包括盆栽、种苗、鳞茎等。为防止物种入侵和病害传播，跨境运输被严禁。"}
{"tag": "种子", "description": "植物的种子或谷物。因涉及物种入侵和农业安全，是各国海关严格禁止或限制入境的物品。"}
{"tag": "化学樟脑丸", "description": "含对二氯苯或萘的化学合成防蛀剂，用于驱虫。其成分对人体有毒性，尤其对儿童构成威胁。"}
{"tag": "可售杀虫杀鼠", "description": "指符合当地法规，允许向公众销售的家用杀虫剂、杀鼠剂等。其成分和标签受到严格监管。"}
{"tag": "蚊香", "description": "通过加热或燃烧，释放驱蚊或杀蚊化学物质的产品。属于农药产品，其有效成分受到严格监管。"}
{"tag": "粘胶型虫鼠诱捕", "description": "涂有强力胶水的板状物，通过物理方式粘住苍蝇、老鼠等害虫。部分地区对粘鼠板有限制。"}
{"tag": "日化-非抑菌", "description": "不含抗菌、消毒成分的普通日用化学品，如常规洗衣液、洗洁精等，主要功能为清洁去污。"}
{"tag": "日化清洁品（韩国）", "description": "在韩国生产或销售的日用化学清洁产品。需符合韩国化学品注册与评估法案（K-REACH）等法规。"}
{"tag": "日化-抑菌", "description": "含有消毒、杀菌或抑菌成分的日用化学品，如消毒液、抗菌洗手液等。其成分和功效宣称受监管。"}
{"tag": "驱虫项圈", "description": "含有药物，佩戴在宠物颈部用于驱杀跳蚤、虱子等体外寄生虫的项圈。属于宠物药品，受严格监管。"}
{"tag": "驱蚊手环", "description": "含有植物精油或化学驱蚊胺，佩戴在手腕上用于驱赶蚊虫的腕带。其有效性和安全性需经评估。"}
{"tag": "驱蚊贴", "description": "含有驱蚊成分，可粘贴在衣物或家具上的贴纸。其有效成分和释放率是产品安全和效果的关键。"}
{"tag": "驱蚊液/乳液", "description": "直接涂抹于皮肤，通过化学成分（如避蚊胺DEET）形成保护层以防止蚊虫叮咬的液体或乳液。"}
{"tag": "其他净水片", "description": "用于非饮用水消毒的化学片剂，如游泳池、鱼缸消毒片。其主要成分通常为含氯化合物。"}
{"tag": "日用酒精棉片/酒精湿巾", "description": "含有医用酒精，用于皮肤或物体表面消毒的一次性棉片或湿巾。属于医疗或消毒产品，受法规监管。"}
{"tag": "消毒剂", "description": "用于杀灭物体表面或环境中微生物的化学制剂，如衣物消毒液、环境消毒剂等。"}
{"tag": "消毒湿巾", "description": "含有消毒成分，用于擦拭手部或物体表面以达到消毒目的的湿巾。其杀菌效果和成分需达标。"}
{"tag": "泳池氯片", "description": "主要成分为三氯异氰尿酸（TCCA），缓慢溶解于水中释放氯气，用于泳池水消毒和维持水质。"}
{"tag": "化肥", "description": "为植物提供一种或多种必需营养元素的化学物质。跨境运输可能受农业和环保法规限制。"}
{"tag": "植物生长调节剂", "description": "用于调节植物生长发育的合成化学物质，如生根粉、催熟剂等。属于农药范畴，受严格监管。"}
{"tag": "对讲机及配件", "description": "利用无线电信号进行短距离通信的手持设备及其专用配件。其频率和功率受各国法规管制。"}
{"tag": "专业防毒面罩", "description": "配备专业滤毒盒，用于防护有毒气体、蒸气和颗粒物的呼吸保护装备。多用于工业或应急场景。"}
{"tag": "非法水烟壶", "description": "经过改装或专门设计，用于吸食大麻油、冰毒等毒品的水烟壶。通常有异常的加热装置或结构。"}
{"tag": "非法烟斗", "description": "结构异常，适用于加热和吸食冰毒、可卡因等毒品的烟斗，如玻璃管状烟斗。"}
{"tag": "电子烟及配件", "description": "通过加热烟油产生气雾供人吸食的电子设备，及其烟弹、雾化器等配件。各国对此类产品监管严格。"}
{"tag": "烟草烟丝", "description": "经过加工的烟草叶制品，可用于卷烟或烟斗。属于特许经营商品，受全球各国严格的法律管制。"}
{"tag": "阿拉伯水烟壶及配件", "description": "用于吸食水烟膏的传统器具，由烟碗、烟管、玻璃瓶和软管组成。其配件如烟碗、吹嘴等亦受管制。"}
{"tag": "卷烟器具", "description": "用于手动制作香烟的工具，如卷烟器、滤嘴棒、卷烟盘等。与烟草制品密切相关，销售受限。"}
{"tag": "卷烟纸/烟卷", "description": "用于包裹烟丝制作手卷烟的专用薄纸。作为烟草制品的一部分，其销售受到严格管制。"}
{"tag": "可售烟斗及配件", "description": "设计用于合法吸食烟斗丝的传统烟斗，材质多为石楠木。其配件如滤芯、清洁工具等一同销售。"}
{"tag": "水烟碳", "description": "专用于加热水烟膏的木炭或椰壳炭块。因其助燃特性，在运输和销售上可能受到限制。"}
{"tag": "雪茄配件", "description": "用于辅助品吸雪茄的工具，主要指雪茄剪和雪茄钻，用于在吸食前打开雪茄头部。"}
{"tag": "烟草研磨器", "description": "用于将干燥的烟草叶研磨成细丝的工具。因其常被用于研磨大麻，在部分地区销售受限。"}
{"tag": "烟盒/烟灰缸", "description": "用于存放香烟的盒子或盛放烟灰的器皿。若商品图片或描述中出现香烟，则会受到销售限制。"}
{"tag": "烟嘴及配件", "description": "安装在香烟前端，用于过滤部分焦油的辅助工具。其配件如清洁刷等也属于受限范围。"}
{"tag": "烟草图案形状禁上mx", "description": "商品本身或其包装、宣传图上带有大麻叶、香烟、吸烟动作等图案，在墨西哥等国禁止销售。"}
{"tag": "减肥贴", "description": "宣称贴于皮肤即可燃烧脂肪、实现减肥效果的贴片。其功效缺乏科学依据，可能引起皮肤过敏。"}
{"tag": "药品", "description": "用于预防、治疗、诊断人类疾病的物质，包括处方药和非处方药。其销售受各国药监部门严格管制。"}
{"tag": "亮片/亮粉/微珠", "description": "用于装饰的塑料小亮片、闪粉或微珠。因造成微塑料污染，在部分国家和地区被禁止销售。"}
{"tag": "湿巾", "description": "预先用液体浸润的无纺布，用于擦拭清洁。根据成分不同，可分为普通湿巾和消毒湿巾。"}
{"tag": "特殊塑料材质用品", "description": "指含有特定受限化学物质（如BPA、邻苯二甲酸盐）或不可降解微塑料的塑料制品。"}
{"tag": "一次性淋膜类纸质餐具", "description": "内壁涂有塑料（PE）淋膜以防水防油的一次性纸杯、纸碗等。因其难以降解回收而受到限制。"}
{"tag": "一次性食品包装", "description": "用于包装食品的一次性塑料盒、保鲜膜、塑料袋等。部分材质和类型因环保问题受限。"}
{"tag": "一次性塑料（棉签/气球棒）", "description": "指棉签的杆或气球的支撑杆为塑料材质。因其体积小、难回收，在许多地区已被禁止。"}
{"tag": "一次性塑料餐具", "description": "一次性使用的塑料刀、叉、勺、盘、吸管等。因造成塑料污染，在全球范围内受到越来越严格的限制。"}
{"tag": "一次性塑料架托", "description": "用于支撑或固定商品（尤其是食品）的一次性塑料托盘或支架。因环保问题而受销售限制。"}
{"tag": "一次性塑料搅拌棒", "description": "用于搅拌咖啡、饮料等的一次性塑料棒。因其非必要性和造成的塑料污染，在多国被禁用。"}
{"tag": "一次性塑料食品容器", "description": "如发泡聚苯乙烯（EPS）餐盒等一次性塑料食品容器。因其对环境的危害，在多地被禁止使用。"}
{"tag": "EMS微电流美容仪/颈纹仪/电子脸部滚轮", "description": "利用微电流（EMS）技术刺激肌肉，宣称有提拉紧致效果的家用电子美容仪器。"}
{"tag": "穿耳器", "description": "一种利用弹簧压力瞬间将耳钉穿过耳垂的工具。因其非无菌和可能造成组织损伤而存在风险。"}
{"tag": "创可贴", "description": "用于覆盖小伤口的无菌敷料，由胶带和吸水垫组成。属于基础医疗用品，用于临时止血保护。"}
{"tag": "激光脱毛仪", "description": "利用激光或强脉冲光（IPL）破坏毛囊，以达到永久或半永久脱毛效果的家用仪器。"}
{"tag": "数字体温计", "description": "通过电子传感器测量体温，并在数字显示屏上读数的设备。比传统水银体温计更安全、快速。"}
{"tag": "纹身针 纹身套装 纹身枪仪器 纹身喷嘴 纹身握把", "description": "用于将墨水刺入皮肤以制作永久性纹身的专业设备及配件。因其侵入性，对无菌要求极高。"}
{"tag": "婴儿洗鼻器", "description": "用于吸出婴幼儿鼻腔内分泌物的辅助工具，帮助缓解鼻塞。分为口吸式、手动泵式和电动式。"}
{"tag": "美瞳", "description": "带有颜色或图案，用于改变虹膜外观的平光或有度数隐形眼镜。属于医疗器械，需谨慎选购。"}
{"tag": "隐形眼镜盒", "description": "用于储存和浸泡隐形眼镜的专用密闭容器。需要定期清洁和更换，以防细菌滋生导致眼部感染。"}
{"tag": "LED牙齿美白灯", "description": "配合美白凝胶使用，通过发出蓝光加速凝胶反应，以去除牙齿表面色渍的家用仪器。"}
{"tag": "电动牙刷", "description": "通过电动机芯的高速振动或旋转，驱动刷头清洁牙齿的牙刷。比手动牙刷清洁效率更高。"}
{"tag": "舌刮", "description": "用于刮除舌苔，清洁舌面，以减少口腔异味和细菌的口腔护理工具。"}
{"tag": "冲牙器", "description": "通过喷射高压脉冲水流，清洁牙缝和牙龈沟等牙刷难以触及区域的口腔护理设备。"}
{"tag": "牙镜", "description": "带有小镜子和长柄的工具，用于检查口腔内不易直接看到的部位，如牙齿内侧。"}
{"tag": "牙镊", "description": "用于夹取和放置口腔内小物品（如棉球）的镊子，是常见的牙科检查工具之一。"}
{"tag": "LED面罩", "description": "内置不同颜色LED灯珠的面罩形美容仪，宣称不同光波可治疗痤疮、促进胶原蛋白生成等。"}
{"tag": "离子导入导出仪器", "description": "利用正负电离子同性相斥、异性相吸的原理，宣称可深层清洁或帮助护肤品吸收的美容仪。"}
{"tag": "电疗仪", "description": "通过电极向人体施加特定低频电流，用于缓解疼痛或刺激肌肉的理疗设备。属医疗器械范畴。"}
{"tag": "电子挖耳勺", "description": "前端配备摄像头和LED灯，可通过手机App实时观察耳道内部情况的可视化挖耳工具。"}
{"tag": "吸奶器", "description": "用于从乳房中吸取母乳的工具，分为手动和电动两种。是哺乳期母亲常用的辅助用品。"}
{"tag": "止鼾器", "description": "通过物理方式（如扩张鼻腔、固定下颚）或微电刺激，旨在改善呼吸道通畅、减少打鼾的设备。"}
{"tag": "女性卫生用品 棉条", "description": "置入阴道内吸收经血的棉质圆柱体。使用时需注意及时更换，以防中毒性休克综合征。"}
{"tag": "卫生巾", "description": "贴于内裤内侧，用于吸收女性经期流出经血的棉质或无纺布制品。是经期最常用的卫生用品。"}
{"tag": "月经杯", "description": "由医用硅胶或乳胶制成，置于阴道内收集经血的杯状物。可重复使用，是一种环保的经期用品。"}
{"tag": "拔罐器", "description": "利用负压吸附在皮肤表面，造成局部充血以达到理疗目的的工具。常见有真空抽气式和传统火罐。"}
{"tag": "床边扶手", "description": "安装在床边，用于辅助行动不便者起身、上下床，或防止从床上坠落的安全护栏。"}
{"tag": "退热贴", "description": "贴于额头，通过凝胶内水分蒸发带走热量，以达到物理降温、缓解发热不适的辅助用品。"}
{"tag": "血糖仪", "description": "通过采集少量指尖血，快速检测血液中葡萄糖浓度的便携式医疗设备。主要供糖尿病患者使用。"}
{"tag": "血压仪/血压计", "description": "用于测量动脉收缩压和舒张压的医疗设备。是监测心血管健康状况的重要工具。"}
{"tag": "血氧仪", "description": "通过夹在指尖，无创检测血液中氧饱和度（SpO2）和脉率的便携式医疗设备。"}
{"tag": "火药", "description": "一种易燃易爆的混合物，是弹药和烟火的核心成分。属于爆炸品，受最严格的法律管控。"}
{"tag": "射钉弹", "description": "用于射钉枪，通过火药燃爆产生动力将钢钉射入墙体等基材的空包弹。属于爆炸物品，受严格管制。"}
{"tag": "烟花爆竹", "description": "以火药为原料，通过燃烧或爆炸产生光、声、色等效果的娱乐产品。因其危险性而受严格管控。"}
{"tag": "炸药", "description": "在外界能量激发下，能产生剧烈化学反应并生成大量气体，造成爆炸的物质。属绝对违禁品。"}
{"tag": "充电宝", "description": "集储电、升压、充电管理于一体的便携式设备，用于为手机等数码产品充电。航空运输有容量限制。"}
{"tag": "纯锂电池", "description": "单独运输的锂金属电池或锂离子电池，非安装在设备内。因其燃爆风险，被列为危险品，运输受限。"}
{"tag": "打火器（无燃气）", "description": "指通过电弧、电热丝等方式点火，但不含可燃气体的点火装置。如电弧打火机、点烟器等。"}
{"tag": "电子打火器", "description": "通过压电效应或电池驱动产生电火花或电弧，用于点燃燃气的装置。常见于燃气灶、热水器。"}
{"tag": "燃气打火机", "description": "内部填充丁烷等可燃气体，用于点火的便携工具。因其易燃易爆特性，被列为航空运输危险品。"}
{"tag": "卡式炉", "description": "使用卡式气罐作为燃料的便携式燃气灶。因涉及燃气和压力容器，其安全标准要求严格。"}
{"tag": "礼花筒", "description": "通过扭动或按压释放压缩气体，将内置的彩纸、亮片等喷出的庆祝用品。不含火药。"}
{"tag": "气罐", "description": "装有压缩或液化气体的便携式金属容器，如丁烷气瓶等。属压力容器，易燃易爆，运输受限。"}
{"tag": "压罐喷雾", "description": "将内容物与抛射剂一同密封在金属罐中的产品。使用时呈雾状喷出，属易燃易爆品。"}
{"tag": "烟雾弹", "description": "通过化学反应产生大量有色或无色烟雾的装置。常用于信号、演习或伪装，属违禁品。"}
{"tag": "打火石、打火棒", "description": "利用金属刮擦产生火花，以点燃引火物的户外求生工具。属于易燃固体，运输受限。"}
{"tag": "固体酒精", "description": "将酒精固化后制成的块状燃料，常用于火锅加热。属于易燃固体，运输和存储需注意防火。"}
{"tag": "红磷、白磷", "description": "化学元素磷的两种同素异形体。白磷燃点极低，红磷用于制造火柴，均属危险化学品。"}
{"tag": "火柴", "description": "利用摩擦生热点燃火柴头化学物质的取火工具。属于易燃品，在航空运输中被严格禁止。"}
{"tag": "镁粉", "description": "金属镁的粉末，是高度易燃的物质，燃烧时会产生强光和高温。属于危险化学品。"}
{"tag": "烟饼（片）", "description": "用于影视拍摄或舞台表演，点燃后能产生大量浓烟的道具。属于易燃品，运输受限。"}
{"tag": "重燃蜡烛", "description": "吹灭后能自动复燃的整蛊蜡烛。因其不易熄灭的特性，存在较高的火灾隐患，被多国禁售。"}
{"tag": "煤油", "description": "一种石油分馏产品，可用作燃料或溶剂。属于易燃液体，其运输和销售受危险品法规管制。"}
{"tag": "包围式学步车", "description": "带有轮子，将婴幼儿包围在其中的辅助行走设备。因其移动速度快、易翻倒，存在严重安全隐患。"}
{"tag": "哺乳枕", "description": "用于在哺乳时支撑婴儿和母亲手臂的C形或O形枕头。需注意其材质安全和透气性。"}
{"tag": "吹气工具", "description": "用于给气球、游泳圈等充气的小型手动或电动气泵。需确保其材质安全，无锐利边缘。"}
{"tag": "磁力球01", "description": "由多个强磁性小球组成的成人减压玩具。若被儿童误食，会在消化道内相互吸引，造成肠道穿孔。"}
{"tag": "带小颗粒的婴儿床品", "description": "带有小珠子、亮片、流苏等小部件装饰的婴儿被褥。这些小部件易脱落，有导致婴儿窒息的风险。"}
{"tag": "儿童充气颈部浮漂", "description": "套在婴幼儿脖子上，使其头部浮在水面的游泳辅助工具。存在漏气、翻覆导致溺水的极高风险。"}
{"tag": "仿真食物", "description": "外形、气味、颜色酷似食物，但不可食用的玩具或装饰品。易被儿童误食，造成窒息或中毒。"}
{"tag": "飞镖指尖陀螺", "description": "将指尖陀螺的叶片设计成飞镖或刀刃形状的玩具。旋转时如同利刃，极易造成割伤，非常危险。"}
{"tag": "非网状床围", "description": "安装在婴儿床内侧，用于防止碰撞的厚实软垫床围。因其不透气，有导致婴儿窒息的严重风险。"}
{"tag": "户外玩具枪（BB弹/橡胶弹）塑料射击玩具凝胶枪凝胶球爆弹弹药", "description": "发射BB弹、凝胶弹等弹丸的玩具枪。其发射动能可能超过安全标准，对眼睛等脆弱部位造成伤害。"}
{"tag": "火焰图案", "description": "指童装上印有火焰、燃烧等图案。在某些文化或安全标准中，此类图案可能被认为不适宜。"}
{"tag": "颗粒牙胶", "description": "表面有多个小颗粒，或由小颗粒串成的婴儿牙胶。颗粒易脱落，存在导致婴儿吞食窒息的风险。"}
{"tag": "蘑菇形婴儿磨牙器或安抚奶嘴", "description": "蘑菇形状的婴儿牙胶或安抚奶嘴。其“菌盖”部分过大，可能完全堵塞婴幼儿口腔，造成窒息。"}
{"tag": "吸水凝珠（胶囊）", "description": "遇水会膨胀数倍的彩色小珠，用作玩具或装饰。若被儿童误食，会在体内膨胀，造成肠梗阻。"}
{"tag": "婴儿包裹类睡眠用品", "description": "如婴儿睡巢、床中床等，将婴儿包裹在柔软平面中的睡眠用品。存在因睡姿不当导致的窒息风险。"}
{"tag": "婴儿汽车座椅", "description": "专为婴幼儿设计的乘车安全座椅。其设计、制造和安装需符合严格的安全标准，以提供有效保护。"}
{"tag": "婴儿襁褓", "description": "用于包裹新生儿，模仿子宫环境以提供安全感的包巾。需采用正确包裹方法，以防影响髋关节发育。"}
{"tag": "婴儿倾斜睡眠产品", "description": "让婴儿以倾斜姿势睡觉的摇床或躺椅。因易导致婴儿头部前倾压迫气管，有极高的窒息风险。"}
{"tag": "婴儿睡袋", "description": "可穿戴的袋状寝具，用于替代传统被子，防止婴儿踢被受凉。需选择合适尺寸以防窒息。"}
{"tag": "婴儿睡眠陪伴玩偶", "description": "放置在婴儿床中陪伴睡眠的毛绒玩具等。柔软的物体会增加婴儿猝死综合征（SIDS）的风险。"}
{"tag": "婴儿睡眠头枕", "description": "宣称可固定婴儿睡姿或矫正头型的枕头。不仅无效，还会增加婴儿在睡眠中窒息的风险。"}
{"tag": "婴儿毯", "description": "用于覆盖婴儿的毯子。在婴儿睡眠时使用需格外小心，避免覆盖口鼻，以降低SIDS风险。"}
{"tag": "婴儿蚊帐床", "description": "自带蚊帐的便携式婴儿床。若带有厚软垫或枕头，可能增加婴儿睡眠时的窒息风险。"}
{"tag": "婴儿自助喂食类商品", "description": "如将奶瓶固定在婴儿嘴边的喂奶枕。在无人看管时，可能导致婴儿呛奶甚至窒息，非常危险。"}
{"tag": "带电击功能的手电筒", "description": "外观为普通手电筒，但前端隐藏有电击触点，可产生高压电弧。属于伪装型电击武器。"}
{"tag": "电击器/电击棒", "description": "通过高压电流使人瞬间麻痹的防身或攻击性武器。属于警用管制器械，在多数国家被严格禁止。"}
{"tag": "警棍/甩棍", "description": "可伸缩的金属短棍，甩出后锁定，用作击打武器。通常为警用装备，属于民间违禁品。"}
{"tag": "警用喷雾", "description": "含有辣椒素等刺激性化学物质，可喷射使目标暂时失能的喷雾。属于管制类防卫武器。"}
{"tag": "手铐、拇指烤、脚镣", "description": "用于束缚人手腕、拇指或脚踝的金属戒具。是执法专用器械，严禁在民用市场非法销售。"}
{"tag": "指虎铁莲花", "description": "套在手指上，用于增强拳头杀伤力的金属武器。属于法律严格管制的攻击性器械，严禁销售。"}
{"tag": "成人用品", "description": "指用于成人性健康、情趣或辅助的器具、服饰等。其销售受到年龄限制和各国法规的约束。"}
{"tag": "刀具", "description": "泛指所有带刃的工具，如菜刀、水果刀等。特定类型（如弹簧刀、超长刀具）会受到严格管制。"}
{"tag": "电动滑板车", "description": "由电力驱动，带车把和站立平台的个人代步工具。其上路行驶在各国受到不同的法律法规限制。"}
{"tag": "电动自行车", "description": "配备电动机和脚踏板，可提供电力辅助的两轮车。其速度、功率等需符合当地交通法规。"}
{"tag": "激光笔", "description": "发射可见激光束的便携式指示工具。大功率激光笔可对人眼或飞机造成危害，受严格功率限制。"}
{"tag": "平衡车", "description": "无车把，依靠使用者身体重心变化来控制行进的电动代步工具。其上路行驶在多地受到限制。"}
{"tag": "强电类商品", "description": "直接接入市电（如110V/220V）的电器，如电视、吹风机等。需符合严格的电气安全标准。"}
{"tag": "纽扣电池供电的产品", "description": "使用纽扣或硬币电池供电的商品。需确保电池仓能被有效固定，以防儿童取出误食造成伤害。"}
{"tag": "弱电类商品", "description": "使用电池、USB或低压适配器供电的电子产品，如蓝牙耳机、USB台灯等。"}
{"tag": "手机", "description": "使用蜂窝网络进行通话和数据传输的移动通信设备。其无线电发射需符合各国认证标准。"}
{"tag": "智能门铃", "description": "带有摄像头、麦克风和无线连接功能的门铃，可远程通话和监控。可能涉及隐私和数据安全问题。"}
{"tag": "无线功能类商品", "description": "包含WiFi、蓝牙等无线通信功能的电子产品。其无线电发射频率和功率需符合当地法规。"}
{"tag": "消毒、灭虫、净化功能", "description": "宣称具有消毒、杀菌、灭虫或空气净化等生物效应的产品。其功效和安全性需经权威机构验证。"}
{"tag": "童鞋", "description": "专为儿童设计的鞋履。需关注其材质安全（如甲醛、重金属含量）和物理安全（如小部件脱落）。"}
{"tag": "儿童自行车座椅", "description": "安装在成人自行车上，用于承载儿童的座椅。其结构强度和安全性需符合相关标准，以防坠落。"}
{"tag": "成人电功能服", "description": "内置加热片或降温模块，通过电池供电实现温度调节的成人服装，如发热马甲、降温服。"}
{"tag": "儿童带电服装", "description": "带有LED灯、发声器等电子元件的儿童服装。需确保电池安全、无漏电风险且小部件不易脱落。"}
{"tag": "儿童睡衣", "description": "专为儿童睡眠时穿着的服装。在一些国家，儿童睡衣的面料需满足严格的阻燃性安全标准。"}
{"tag":-": "儿童造型服", "description": "供儿童在派对、节日等场合穿着的角色扮演服装。需注意其材质安全及是否存在窒息风险。"}
{"tag": "美国带绳窗帘", "description": "带有拉绳或珠链的窗帘、百叶窗等。其拉绳易形成绳圈，对儿童有勒颈窒息的严重风险。"}
{"tag": "婴儿睡衣", "description": "专为婴幼儿设计的连体或分体式睡衣。需确保材质柔软、透气，且无小部件脱落风险。"}
{"tag": "痘痘贴", "description": "用于遮盖痘痘或吸收分泌物的水胶体贴片。部分产品宣称的治疗效果可能使其被归类为药品。"}
{"tag": "化妆品", "description": "用于涂抹、喷洒于人体表面，以达到清洁、护肤、美容和修饰目的的日用化学工业产品。"}
{"tag": "水龙头及延伸配件", "description": "用于控制水流的阀门及其延伸装置。与饮用水接触的产品需符合相关的卫生安全标准。"}
{"tag": "玩具气球", "description": "专为儿童玩耍设计的乳胶或铝箔气球。未充气的气球或碎片可能对儿童造成窒息危险。"}
{"tag": "装饰性气球", "description": "主要用于派对、婚礼等场合装饰的气球。尺寸通常较大，或有特殊造型。"}
{"tag": "食品&饮料", "description": "供人类食用或饮用的制成品。其生产、标签和销售受到各国食品安全法规的严格监管。"}
{"tag": "食品＆保健品", "description": "宣称具有特定保健功能的食品。其功能声称、成分和广告受到严格监管，不得宣传治疗效果。"}
{"tag": "普通食品接触", "description": "设计与食品直接接触的非一次性餐具、厨具、容器等。其材质需符合食品级安全标准。"}
{"tag": "图书类", "description": "印刷或数字形式的书籍、杂志、漫画等出版物。内容不得违反当地法律法规。"}
{"tag": "棒马", "description": "一根杆子顶端带有马头造型的儿童骑乘玩具。需确保杆身光滑无毛刺，马头部件牢固。"}
{"tag": "高脚椅及坐垫", "description": "让婴幼儿能与成人同桌吃饭的高脚座椅。其结构稳定性和安全束缚系统至关重要，以防儿童跌落。"}
{"tag": "脚踏凳", "description": "供儿童踩踏以增加高度的小凳子，常用于浴室或厨房。需有防滑表面和稳固的结构。"}
{"tag": "门栏", "description": "安装在楼梯口或门口，用于阻止婴幼儿进入危险区域的安全护栏。其锁定机制必须牢固可靠。"}
{"tag": "学步车", "description": "辅助婴幼儿学习行走的工具。其中，包围式学步车因安全风险高，在多国被禁售。"}
{"tag": "摇摆的马和动物", "description": "儿童可以骑在上面前后摇摆的玩具，如木马。需确保结构稳固，不会侧翻。"}
{"tag": "摇篮床", "description": "可以轻微摇晃以安抚婴儿入睡的小床。需确保摇摆幅度安全，且床体结构稳固。"}
{"tag": "摇椅和秋千", "description": "供婴幼儿坐卧的摇椅或室内秋千。需有可靠的安全带，并确保安装稳固，防止倾倒或坠落。"}
{"tag": "婴儿安全带和牵引绳", "description": "用于在公共场所防止儿童走失的束缚带或牵引绳。其设计需确保儿童舒适且不会被勒伤。"}
{"tag": "婴儿背带和配件", "description": "用于将婴儿背负在身上的工具。需确保其设计能正确支撑婴儿的脊柱和髋部，并防止跌落。"}
{"tag": "婴儿餐具", "description": "专为婴幼儿设计的勺、碗、杯等。需由食品级安全材料制成，且不含BPA等有害物质。"}
{"tag": "婴儿车", "description": "用于载运婴幼儿出行的手推车。其刹车系统、安全带和结构稳定性是核心安全指标。"}
{"tag": "婴儿车配件", "description": "如雨罩、蚊帐、挂钩等用于婴儿车的附加物品。需确保安装后不影响婴儿车的安全性和稳定性。"}
{"tag": "婴儿床", "description": "专供婴幼儿睡觉的床。其栏杆间距、床垫贴合度等需符合严格的安全标准，以防夹伤或窒息。"}
{"tag": "婴儿棉品", "description": "指婴儿用的棉质纺织品，如口水巾、隔尿垫、包被等。需确保材质柔软、无荧光剂。"}
{"tag": "婴儿皮肤护理", "description": "专为婴儿设计的护肤品，如润肤露、护臀霜等。配方需温和无刺激，并通过皮肤测试。"}
{"tag": "婴儿推车储存", "description": "用于整理和存放婴儿推车的挂袋或收纳架。需确保其不影响推车的平衡和安全。"}
{"tag": "婴儿牙刷", "description": "专为婴幼儿设计的牙刷，刷毛柔软，刷头小巧。有指套式和手柄式等类型。"}
{"tag": "婴儿运动和游戏垫", "description": "供婴儿爬行、玩耍的软垫。需由无毒、无味的材料制成，并具有一定的缓冲性能。"}
{"tag": "婴儿座椅配件", "description": "用于婴儿汽车座椅或餐椅的配件，如坐垫、头枕等。需确保不影响座椅原有的安全性能。"}
{"tag": "游戏床", "description": "为婴幼儿提供安全游戏空间的围栏式床。需确保其结构稳固，网布无破损，锁定机制可靠。"}
{"tag": "幼儿床", "description": "从婴儿床过渡到成人床之间使用的儿童床，通常尺寸较小，带有护栏。"}
{"tag": "婴儿用品", "description": "泛指所有专为婴幼儿设计和使用的产品。对此类产品的材料、结构和化学安全有最高要求。"}
{"tag": "磁性画板", "description": "利用磁粉和磁性画笔进行绘画和擦除的玩具画板。需确保磁性画笔不会脱落产生小零件。"}
{"tag": "磁性积木玩具", "description": "内置磁铁，可相互吸附搭建造型的积木。需确保内部磁铁不会脱落，以防被儿童误食。"}
{"tag": "磁性游戏板", "description": "带有磁性棋子或部件的棋盘游戏。需确保磁性部件不易脱落，且磁通量指数符合安全标准。"}
{"tag": "儿童游泳圈", "description": "供儿童在水中漂浮和玩耍的充气圈。属于水上玩具而非救生设备，使用时需全程成人监护。"}
{"tag": "积木玩具", "description": "由不同形状的模块组成，可进行搭建的玩具。针对不同年龄段，对积木的尺寸有不同安全要求。"}
{"tag": "减压软玩具", "description": "手感柔软，可任意揉捏并缓慢回弹的玩具。需注意其材质安全，以及是否易被咬破产生小碎片。"}
{"tag": "毛绒公仔", "description": "由毛绒面料和填充物制成的玩偶。需关注其眼睛等小部件是否牢固，以及填充物是否卫生安全。"}
{"tag": "毛绒挂件吊饰", "description": "小型的毛绒玩偶，带有挂绳或钥匙扣，可挂在书包、钥匙上。需确保小部件不易脱落。"}
{"tag": "水枪", "description": "通过压力喷射水流的玩具枪。需注意其喷射压力是否过大，以及储水仓是否易滋生细菌。"}
{"tag": "玩具泡沫爆破枪", "description": "发射柔软泡沫子弹的玩具枪。需确保子弹材质柔软，发射动能符合安全标准，不会伤及眼睛。"}
{"tag": "纸质拼图", "description": "将一幅图画分割成许多小块，供玩家重新拼合的益智玩具。需注意纸张边缘是否光滑。"}
{"tag": "字母拼图", "description": "以字母形状为拼图块的益智玩具，常用于儿童认知启蒙。需确保拼图块大小对幼儿是安全的。"}
{"tag": "婴童玩具", "description": "专为3岁以下婴幼儿设计的玩具。对此类玩具的物理安全（如小零件、锐利边缘）有最严格的要求。"}
{"tag": "枪支武器相关", "description": "涉及真实枪支、其核心部件、弹药或其制造方法的商品或信息。属于全球范围内的绝对违禁品。"}
{"tag": "暴露款商品", "description": "展示过度裸露或带有性暗示的服装、图片或商品。其销售受到平台政策和地方法规的限制。"}
{"tag": "推荐暴露款", "description": "指包含露骨色情、性行为或极端性暗示的商品，通常在所有平台和地区都受到最严格的禁止。"}
{"tag": "推荐恶心商品", "description": "指包含令人极度不适、反感或恶心内容的商品，如仿真排泄物、密集图案、血腥医疗场景等。"}
{"tag": "推荐恐怖商品", "description": "指包含血腥、暴力、惊悚或恐怖元素，可能引起用户强烈恐惧或不安的商品。"}
{"tag": "纽扣电池", "description": "小型、扁圆形的电池。若被儿童误食，会在体内造成严重的化学灼伤，甚至致命，需妥善保管。"}

##FINAL_REMINDER##
**Multi-tag output is PREFERRED and EXPECTED from ##REFERENCE_CATEGORIES##.**
- Don't hesitate to include multiple relevant categories
- Comprehensive classification serves compliance better than restrictive single tags  
- **Think: "What are ALL the ways this item could be problematic based on ##REFERENCE_CATEGORIES##?"**
- **When in doubt about any category from ##REFERENCE_CATEGORIES##, include rather than exclude**
- Your comprehensive coverage helps human reviewers make informed decisions"""

# system_prompt = "请分析图片。"
user_prompt = "Please classify the item in this image according to the categories defined in the system."

def expand_tags(tag):
    meta_dict = {
      "匕首": [
        "刀具"
      ],
      "超长刀具": [
        "刀具"
      ],
      "弹簧刀": [
        "刀具"
      ],
      "蝴蝶刀": [
        "刀具"
      ],
      "灰鲭鲨刀": [
        "刀具"
      ],
      "伪装刀具": [
        "刀具"
      ],
      "爪刀": [
        "刀具"
      ],
      "弹弓弩": [
        "弹弓"
      ],
      "弓": [
        "武器",
        "射击类武器"
      ],
      "箭/箭头": [
        "弓",
        "弩"
      ],
      "伪装牙签": [
        "武器"
      ],
      "钥匙棍": [
        "枪支武器相关"
      ],
      "仿真枪": [
        "枪支武器相关"
      ],
      "仿真子弹": [
        "枪支武器相关"
      ],
      "弹匣": [
        "枪支武器相关"
      ],
      "弹匣装弹器": [
        "枪支武器相关"
      ],
      "瞄准镜": [
        "枪支武器相关"
      ],
      "枪管": [
        "仿真枪"
      ],
      "枪托": [
        "仿真枪"
      ],
      "枪支导轨": [
        "仿真枪"
      ],
      "枪支握把": [
        "仿真枪"
      ],
      "消音器": [
        "枪支武器相关"
      ],
      "带电击功能的手电筒": [
        "电击器/电击棒"
      ],
      "电子打火器": [
        "打火器（无燃气）"
      ],
      "燃气打火机": [
        "打火器（无燃气）",
        "电子打火器"
      ],
      "打火石、打火棒": [
        "打火器（无燃气）"
      ],
      "红磷、白磷": [
        "打火石、打火棒",
        "固体酒精",
        "火柴",
        "镁粉",
        "火药",
        "射钉弹",
        "烟花爆竹",
        "炸药"
      ],
      "火柴": [
        "打火器具"
      ],
      "镁粉": [
        "打火器（无燃气）",
        "易燃品"
      ],
      "火药": [
        "易燃易爆物品"
      ],
      "射钉弹": [
        "火药"
      ],
      "炸药": [
        "火药"
      ],
      "孔明灯": [
        "烟花爆竹"
      ],
      "礼花筒": [
        "烟花爆竹"
      ],
      "气罐": [
        "压罐喷雾"
      ],
      "吸毒工具": [
        "毒品"
      ],
      "易制毒成分": [
        "毒品"
      ],
      "制毒工具-胶囊填充机": [
        "制毒工具",
        "毒品"
      ],
      "阿拉伯水烟壶及配件": [
        "非法水烟壶",
        "烟草烟丝"
      ],
      "卷烟纸/烟卷": [
        "烟草烟丝",
        "卷烟器具"
      ],
      "可售烟斗及配件": [
        "烟草烟丝"
      ],
      "水烟碳": [
        "阿拉伯水烟壶及配件",
        "烟草烟丝"
      ],
      "雪茄配件": [
        "电子烟及配件",
        "烟草烟丝"
      ],
      "烟盒/烟灰缸": [
        "烟草烟丝",
        "阿拉伯水烟壶及配件",
        "可售烟斗及配件",
        "雪茄配件"
      ],
      "烟嘴及配件": [
        "电子烟及配件",
        "烟草烟丝"
      ],
      "舌刮": [
        "日化清洁品（韩国）",
        "日化-非抑菌",
        "日化-抑菌"
      ],
      "LED面罩": [
        "EMS微电流美容仪/颈纹仪/电子脸部滚轮"
      ],
      "卫生巾": [
        "女性卫生用品"
      ],
      "月经杯": [
        "女性卫生用品"
      ],
      "美瞳": [
        "隐形眼镜盒"
      ],
      "纯锂电池": [
        "充电宝"
      ],
      "纽扣电池供电的产品": [
        "充电宝",
        "纯锂电池",
        "纽扣电池"
      ],
      "对讲机及配件": [
        "无线功能类商品"
      ],
      "电视机顶盒": [
        "弱电类商品",
        "无线功能类商品"
      ],
      "智能门铃": [
        "弱电类商品",
        "无线功能类商品"
      ],
      "无线功能类商品": [
        "弱电类商品",
        "电子产品"
      ],
      "无漏保吹风机/热风梳": [
        "强电类商品"
      ],
      "消费级无人机": [
        "无线功能类商品"
      ],
      "一般宠物食品（猫狗马鱼）": [
        "宠物食品"
      ],
      "驱虫项圈": [
        "宠物药品"
      ],
      "种子": [
        "植物活体"
      ],
      "化学樟脑丸": [
        "可售杀虫杀鼠"
      ],
      "蚊香": [
        "可售杀虫杀鼠"
      ],
      "粘胶型虫鼠诱捕": [
        "可售杀虫杀鼠"
      ],
      "日化清洁品（韩国）": [
        "日化-非抑菌",
        "日化-抑菌"
      ],
      "日化-抑菌": [
        "日化-非抑菌",
        "日化清洁品（韩国）"
      ],
      "其他净水片": [
        "日化-抑菌"
      ],
      "日用酒精棉片/酒精湿巾": [
        "日化-抑菌"
      ],
      "消毒剂": [
        "日化-抑菌"
      ],
      "消毒湿巾": [
        "日化-抑菌",
        "消毒剂"
      ],
      "泳池氯片": [
        "消毒剂"
      ],
      "铁粉加热鞋垫": [
        "有刺伤,划伤隐患的服装",
        "其他安全隐患类服装配饰"
      ],
      "有刺伤,划伤隐患的服装": [
        "其他安全隐患类服装配饰"
      ],
      "其他安全隐患类服装配饰": [],
      "成人电功能服": [
        "成人带电服装"
      ],
      "儿童带电服装": [
        "成人带电服装"
      ],
      "儿童睡衣": [
        "有刺伤,划伤隐患的服装",
        "其他安全隐患类服装配饰"
      ],
      "儿童造型服": [
        "有刺伤,划伤隐患的服装",
        "其他安全隐患类服装配饰"
      ],
      "美国带绳窗帘": [
        "儿童安全隐患类"
      ],
      "婴儿睡衣": [
        "儿童睡衣"
      ],
      "包围式学步车": [
        "学步车"
      ],
      "哺乳枕": [
        "婴儿用品"
      ],
      "带小颗粒的婴儿床品": [
        "婴儿床品",
        "婴儿用品"
      ],
      "儿童充气颈部浮漂": [
        "婴童玩具"
      ],
      "非网状床围": [
        "婴儿床品",
        "婴儿用品"
      ],
      "婴儿包裹类睡眠用品": [
        "婴儿睡袋",
        "婴儿襁褓"
      ],
      "婴儿襁褓": [
        "婴儿用品"
      ],
      "婴儿倾斜睡眠产品": [
        "婴儿用品",
        "婴儿睡眠用品"
      ],
      "婴儿睡袋": [
        "婴儿床品",
        "婴儿用品"
      ],
      "婴儿睡眠陪伴玩偶": [
        "婴童玩具",
        "毛绒公仔"
      ],
      "婴儿睡眠头枕": [
        "婴儿用品"
      ],
      "婴儿毯": [
        "婴儿床品",
        "婴儿用品"
      ],
      "婴儿蚊帐床": [
        "婴儿用品",
        "婴儿床"
      ],
      "婴儿自助喂食类商品": [
        "婴儿用品"
      ],
      "吹气工具": [
        "婴童玩具"
      ],
      "磁力球01": [
        "玩具"
      ],
      "仿真食物": [
        "婴童玩具"
      ],
      "飞镖指尖陀螺": [
        "玩具",
        "婴童玩具"
      ],
      "户外玩具枪（BB弹/橡胶弹）塑料射击玩具凝胶枪凝胶球爆弹弹药": [
        "婴童玩具"
      ],
      "火焰图案": [
        "玩具"
      ],
      "颗粒牙胶": [
        "婴童玩具"
      ],
      "蘑菇形婴儿磨牙器或安抚奶嘴": [
        "婴童玩具",
        "婴儿用品"
      ],
      "吸水凝珠（胶囊）": [
        "婴童玩具"
      ],
      "磁性画板": [
        "儿童玩具",
        "玩具"
      ],
      "磁性积木玩具": [
        "积木玩具",
        "婴童玩具"
      ],
      "磁性游戏板": [
        "玩具"
      ],
      "儿童游泳圈": [
        "婴童玩具"
      ],
      "积木玩具": [
        "婴童玩具"
      ],
      "减压软玩具": [
        "玩具"
      ],
      "毛绒公仔": [
        "婴童玩具"
      ],
      "毛绒挂件吊饰": [
        "玩具"
      ],
      "水枪": [
        "玩具"
      ],
      "玩具泡沫爆破枪": [
        "婴童玩具"
      ],
      "纸质拼图": [
        "玩具"
      ],
      "字母拼图": [
        "玩具"
      ],
      "玩具气球": [
        "婴童玩具"
      ],
      "棒马": [
        "婴童玩具"
      ],
      "高脚椅及坐垫": [
        "婴儿用品"
      ],
      "脚踏凳": [
        "婴儿用品"
      ],
      "门栏": [
        "婴童玩具"
      ],
      "学步车": [
        "包围式学步车"
      ],
      "摇摆的马和动物": [
        "婴童玩具"
      ],
      "摇篮床": [
        "婴儿用品"
      ],
      "摇椅和秋千": [
        "婴童玩具"
      ],
      "婴儿安全带和牵引绳": [
        "婴儿用品"
      ],
      "婴儿背带和配件": [
        "婴儿用品"
      ],
      "婴儿车": [],
      "婴儿车配件": [
        "婴儿用品"
      ],
      "婴儿床": [
        "婴童玩具",
        "婴儿用品"
      ],
      "婴儿推车储存": [
        "婴儿用品"
      ],
      "婴儿座椅配件": [],
      "游戏床": [
        "婴儿用品"
      ],
      "幼儿床": [
        "婴儿用品"
      ],
      "婴儿餐具": [
        "婴儿用品"
      ],
      "婴儿棉品": [
        "婴儿用品"
      ],
      "婴儿皮肤护理": [
        "婴儿用品"
      ],
      "婴儿牙刷": [
        "婴儿用品"
      ],
      "婴儿运动和游戏垫": [
        "婴儿用品"
      ],
      "一次性塑料架托": [
        "一次性塑料餐具",
        "一次性塑料食品容器"
      ],
      "一次性塑料搅拌棒": [
        "一次性塑料餐具"
      ],
      "一次性塑料食品容器": [
        "一次性食品包装"
      ],
      "可移动酒精壁炉": [
        "固体酒精"
      ],
      "食品&饮料": [
        "食品＆保健品"
      ],
      "食品＆保健品": [
        "食品&饮料"
      ],
      "壮阳增大产品": [
        "成人用品"
      ],
      "成人用品": [
        "壮阳增大产品",
        "插入式导管"
      ],
      "推荐暴露款": [
        "暴露款商品"
      ],
      "痘痘贴": [
        "化妆品"
      ]
    }

    parent_similar_tag_list = meta_dict.get(tag, [])
    parent_similar_tags = ",".join(parent_similar_tag_list)

    return f"{tag},{parent_similar_tags}".strip(',')

def create_record(tags, s3_prefix, filename):
    s3_path = f"s3://{s3_bucket}/{s3_prefix}/{filename}"

    img_format = "jpeg" if filename.endswith(".jpg") else filename.split('.')[-1]

    if img_format not in ['jpeg']:
        # print(img_format)
        return None
    
    return {
        "schemaVersion": "bedrock-conversation-2024",
        "system": [{
            "text": system_prompt
        }],
        "messages": [{
                "role": "user",
                "content": [
                    {
                        "image": {
                            "format": img_format,
                            "source": {
                                "s3Location": {
                                    "uri": s3_path,
                                    "bucketOwner": account_id
                                }
                            }
                        }
                    },
                    {	"text": user_prompt	}
                ]
            },
            {
                "role": "assistant",
                "content": [{
                    "text": tags
                }]
            }
        ]
    }


### prepare training dataset

In [2]:
# Load data
version = 'v1'

df = pd.read_excel('data/shein_data/train_data_balanced.xlsx')

s3_prefix = "projects/nova-image-tagging/data/shein_data/images"

train_count = 0
# Generate JSONL
train_data_path = f"data/merged_data/{version}/nova_sft_trainset.jsonl"
with open(train_data_path, 'w', encoding='utf-8') as f:
    for idx, row in df.iterrows():
        tags = expand_tags(row['flag'])
        gt_label = '{"result":"' + tags + '"}'
        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            train_count += 1

print(f"Generated {train_count} records in nova_sft_dataset.jsonl")

Generated 1418 records in nova_sft_dataset.jsonl


### Test set

In [3]:
# Load data
df = pd.read_excel('data/shein_data/test_data.xlsx')

s3_prefix = "projects/nova-image-tagging/data/shein_data/images"
test_count = 0
# Generate JSONL
with open(f'data/merged_data/{version}/nova_sft_testset.jsonl', 'w', encoding='utf-8') as f:
    for idx, row in df.iterrows():
        tags = expand_tags(row['flag'])
        gt_label = '{"result":"' + tags + '"}'
        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            test_count += 1

print(f"Generated {test_count} records in nova_sft_dataset.jsonl")

Generated 434 records in nova_sft_dataset.jsonl


### Using Product10K dataset

In [4]:
product_path = "data/Product10K/test_wt_group_label.csv"

product_df = pd.read_csv(product_path)
print(product_df.shape)
product_df.head()

(55376, 5)


,filename,class,Usage,group,flag
0,3963142.jpg,0,Public,0,服装类、男装、时尚服饰
1,1209854.jpg,0,Public,0,服装类、男装、时尚服饰
2,8638678.jpg,0,Private,0,服装类、男装、时尚服饰
3,6997109.jpg,0,Private,0,服装类、男装、时尚服饰
4,3134887.jpg,0,Private,0,服装类、男装、时尚服饰


In [5]:
product_df['flag'] = product_df['flag'].apply(lambda x: x.replace('、', ','))
product_df.head()

,filename,class,Usage,group,flag
0,3963142.jpg,0,Public,0,"服装类,男装,时尚服饰"
1,1209854.jpg,0,Public,0,"服装类,男装,时尚服饰"
2,8638678.jpg,0,Private,0,"服装类,男装,时尚服饰"
3,6997109.jpg,0,Private,0,"服装类,男装,时尚服饰"
4,3134887.jpg,0,Private,0,"服装类,男装,时尚服饰"


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def hybrid_stratified_sampling(test_df, total_samples, train_samples, test_samples, 
                             stratify_column='label', random_state=42):
    """
    Hybrid approach: stratified for large classes, random for small classes
    Keeps all classes including those with only 1 sample
    """
    
    print("=== Hybrid Stratified Sampling ===")
    print(f"Original data shape: {test_df.shape}")
    print(f"Target total samples: {total_samples}")
    print(f"Target train samples: {train_samples}")
    print(f"Target test samples: {test_samples}")
    
    # Analyze class distribution
    class_counts = test_df[stratify_column].value_counts().sort_values(ascending=False)
    small_classes = class_counts[class_counts == 1].index
    large_classes = class_counts[class_counts >= 2].index
    
    print(f"\n=== Class Analysis ===")
    print(f"Total classes: {len(class_counts)}")
    print(f"Small classes (1 sample): {len(small_classes)}")
    print(f"Large classes (>=2 samples): {len(large_classes)}")
    
    if len(small_classes) > 0:
        print(f"Small classes: {list(small_classes)}")
    
    # Show class distribution
    print(f"\n=== Class Distribution ===")
    for cls, count in class_counts.head(10).items():  # Show top 10
        print(f"  {cls}: {count} samples")
    if len(class_counts) > 10:
        print(f"  ... and {len(class_counts) - 10} more classes")
    
    # Handle the special case where all classes are small
    if len(large_classes) == 0:
        print("\n=== All Classes Are Small - Using Pure Random Sampling ===")
        return pure_random_sampling(test_df, total_samples, train_samples, test_samples, random_state)
    
    # Split data into small and large class groups
    small_class_df = test_df[test_df[stratify_column].isin(small_classes)]
    large_class_df = test_df[test_df[stratify_column].isin(large_classes)]
    
    print(f"\n=== Data Split ===")
    print(f"Small class samples: {len(small_class_df)}")
    print(f"Large class samples: {len(large_class_df)}")
    
    # Calculate sampling proportions
    total_original = len(test_df)
    small_class_ratio = len(small_class_df) / total_original
    large_class_ratio = len(large_class_df) / total_original
    
    print(f"Small class ratio: {small_class_ratio:.3f}")
    print(f"Large class ratio: {large_class_ratio:.3f}")
    
    # Calculate target samples for each group
    target_samples_from_small = int(total_samples * small_class_ratio)
    target_samples_from_large = total_samples - target_samples_from_small
    
    # Ensure we don't exceed available samples
    actual_samples_from_small = min(target_samples_from_small, len(small_class_df))
    actual_samples_from_large = min(target_samples_from_large, len(large_class_df))
    
    # Adjust if we can't get enough samples
    actual_total_samples = actual_samples_from_small + actual_samples_from_large
    if actual_total_samples < total_samples:
        print(f"Warning: Can only sample {actual_total_samples} instead of {total_samples}")
        # Try to compensate by taking more from the larger group
        if actual_samples_from_large < len(large_class_df):
            compensation = min(total_samples - actual_total_samples, 
                             len(large_class_df) - actual_samples_from_large)
            actual_samples_from_large += compensation
            actual_total_samples += compensation
    
    print(f"\n=== Sampling Plan ===")
    print(f"From small classes: {actual_samples_from_small} samples")
    print(f"From large classes: {actual_samples_from_large} samples")
    print(f"Total: {actual_samples_from_small + actual_samples_from_large} samples")
    
    # Sample from small classes (randomly, since we can't stratify single samples)
    sampled_small = pd.DataFrame()
    if actual_samples_from_small > 0 and len(small_class_df) > 0:
        print(f"\nSampling {actual_samples_from_small} from small classes (random sampling)...")
        sampled_small = small_class_df.sample(n=actual_samples_from_small, random_state=random_state)
        print(f"Small classes sampled: {len(sampled_small)} samples")
        small_class_distribution = sampled_small[stratify_column].value_counts()
        print("Small classes in sample:", dict(small_class_distribution))
    
    # Sample from large classes (with stratification)
    sampled_large = pd.DataFrame()
    if actual_samples_from_large > 0 and len(large_class_df) > 0:
        print(f"\nSampling {actual_samples_from_large} from large classes (stratified sampling)...")
        if actual_samples_from_large >= len(large_class_df):
            sampled_large = large_class_df.copy()
            print(f"Taking all {len(sampled_large)} samples from large classes")
        else:
            try:
                sampled_large, _ = train_test_split(
                    large_class_df,
                    train_size=actual_samples_from_large,
                    stratify=large_class_df[stratify_column],
                    random_state=random_state
                )
                print(f"Large classes sampled: {len(sampled_large)} samples (stratified)")
            except ValueError as e:
                print(f"Stratified sampling failed for large classes: {e}")
                print("Falling back to random sampling for large classes...")
                sampled_large = large_class_df.sample(n=actual_samples_from_large, random_state=random_state)
                print(f"Large classes sampled: {len(sampled_large)} samples (random)")
    
    # Combine samples
    sample_pieces = [df for df in [sampled_small, sampled_large] if not df.empty]
    
    if not sample_pieces:
        raise ValueError("No samples were selected!")
    
    sampled_df = pd.concat(sample_pieces, ignore_index=True)
    
    # Shuffle the combined dataset
    sampled_df = sampled_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    print(f"\n=== Sampling Results ===")
    print(f"Total sampled: {len(sampled_df)} samples")
    
    # Show final class distribution in sampled data
    final_class_counts = sampled_df[stratify_column].value_counts()
    print(f"Classes in sampled data: {len(final_class_counts)}")
    
    # Adjust train/test split sizes if needed
    actual_total = len(sampled_df)
    if train_samples + test_samples > actual_total:
        print(f"\nAdjusting train/test split (requested {train_samples + test_samples}, have {actual_total})")
        ratio = train_samples / (train_samples + test_samples)
        train_samples = int(actual_total * ratio)
        test_samples = actual_total - train_samples
        print(f"Adjusted: train={train_samples}, test={test_samples}")
    
    # Determine if we can use stratified split for train/test
    final_class_counts = sampled_df[stratify_column].value_counts()
    classes_too_small = final_class_counts[final_class_counts < 2]
    can_stratify = len(classes_too_small) == 0
    
    print(f"\n=== Train/Test Split ===")
    if can_stratify:
        print("Using stratified split for train/test")
        try:
            train_df, test_df_final = train_test_split(
                sampled_df,
                train_size=train_samples,
                test_size=test_samples,
                stratify=sampled_df[stratify_column],
                random_state=random_state
            )
        except ValueError as e:
            print(f"Stratified split failed: {e}")
            print("Falling back to random split")
            can_stratify = False
    
    if not can_stratify:
        print(f"Using random split for train/test ({len(classes_too_small)} classes have <2 samples)")
        if len(classes_too_small) > 0:
            print(f"Classes with <2 samples: {list(classes_too_small.index)}")
        
        train_df, test_df_final = train_test_split(
            sampled_df,
            train_size=train_samples,
            test_size=test_samples,
            random_state=random_state
        )
    
    print(f"\n=== Final Results ===")
    print(f"Train set: {len(train_df)} samples")
    print(f"Test set: {len(test_df_final)} samples")
    print(f"Total: {len(train_df) + len(test_df_final)} samples")
    
    return train_df, test_df_final, sampled_df

def pure_random_sampling(test_df, total_samples, train_samples, test_samples, random_state=42):
    """
    Fallback function for when all classes are too small for stratification
    """
    print("Using pure random sampling (no stratification possible)")
    
    # First sample
    if total_samples >= len(test_df):
        sampled_df = test_df.copy()
    else:
        sampled_df = test_df.sample(n=total_samples, random_state=random_state)
    
    # Adjust split sizes if needed
    actual_total = len(sampled_df)
    if train_samples + test_samples > actual_total:
        ratio = train_samples / (train_samples + test_samples)
        train_samples = int(actual_total * ratio)
        test_samples = actual_total - train_samples
    
    # Split
    train_df, test_df_final = train_test_split(
        sampled_df,
        train_size=train_samples,
        test_size=test_samples,
        random_state=random_state
    )
    
    return train_df, test_df_final, sampled_df

def validate_results(original_df, sampled_df, train_df, test_df_final, stratify_column='label'):
    """
    Validate and report on the sampling results
    """
    print("\n" + "="*50)
    print("VALIDATION REPORT")
    print("="*50)
    
    # Basic counts
    print(f"Original data: {len(original_df)} samples, {original_df[stratify_column].nunique()} classes")
    print(f"Sampled data: {len(sampled_df)} samples, {sampled_df[stratify_column].nunique()} classes")
    print(f"Train set: {len(train_df)} samples, {train_df[stratify_column].nunique()} classes")
    print(f"Test set: {len(test_df_final)} samples, {test_df_final[stratify_column].nunique()} classes")
    
    # Class distribution comparison
    orig_dist = original_df[stratify_column].value_counts(normalize=True).sort_index()
    sampled_dist = sampled_df[stratify_column].value_counts(normalize=True).sort_index()
    
    print(f"\n=== Class Distribution Comparison (Top 10) ===")
    comparison_df = pd.DataFrame({
        'Original_Count': original_df[stratify_column].value_counts(),
        'Original_Prop': orig_dist,
        'Sampled_Count': sampled_df[stratify_column].value_counts(),
        'Sampled_Prop': sampled_dist
    }).fillna(0)
    
    comparison_df['Prop_Diff'] = abs(comparison_df['Original_Prop'] - comparison_df['Sampled_Prop'])
    comparison_df = comparison_df.sort_values('Original_Count', ascending=False)
    
    print(comparison_df.head(10).round(4))
    
    # Check for large deviations
    large_deviations = comparison_df[comparison_df['Prop_Diff'] > 0.05]
    if len(large_deviations) > 0:
        print(f"\n⚠️  Warning: {len(large_deviations)} classes have >5% proportion deviation")
        print(large_deviations[['Original_Prop', 'Sampled_Prop', 'Prop_Diff']].head())
    else:
        print("\n✅ All class proportions are well preserved (<5% deviation)")
    
    # Check for missing classes
    missing_classes = set(orig_dist.index) - set(sampled_dist.index)
    if missing_classes:
        print(f"\n⚠️  Warning: {len(missing_classes)} classes missing from sample: {list(missing_classes)[:5]}...")
    else:
        print("\n✅ All classes preserved in sample")

In [7]:
total_train = 20000
total_test = 1000

train_samples = total_train - train_count
test_samples = total_test - test_count 
total_samples = train_samples + test_samples

print("CONFIGURATION:")
print(f"train_count: {train_count}")
print(f"test_count: {test_count}")
print(f"train_samples needed: {train_samples}")
print(f"test_samples needed: {test_samples}")
print(f"total_samples needed: {total_samples}")

# Execute the hybrid stratified sampling
try:
    train_df_sample, test_df_sample, sampled_df = hybrid_stratified_sampling(
        product_df, 
        total_samples, 
        train_samples, 
        test_samples,
        stratify_column='flag',
        random_state=42
    )
    
    # Validate results
    validate_results(product_df, sampled_df, train_df_sample, test_df_sample, 'flag')
    
    print(f"\n🎉 SUCCESS!")
    print(f"Final train set: {len(train_df_sample)} samples")
    print(f"Final test set: {len(test_df_sample)} samples")
    
    # Show final class distributions
    print(f"\nTrain set class distribution:")
    print(train_df_sample['flag'].value_counts().head())
    
    print(f"\nTest set class distribution:")
    print(test_df_sample['flag'].value_counts().head())
    
except Exception as e:
    print(f"❌ Error occurred: {e}")
    import traceback
    traceback.print_exc()


CONFIGURATION:
train_count: 1418
test_count: 434
train_samples needed: 18582
test_samples needed: 566
total_samples needed: 19148
=== Hybrid Stratified Sampling ===
Original data shape: (55376, 5)
Target total samples: 19148
Target train samples: 18582
Target test samples: 566

=== Class Analysis ===
Total classes: 295
Small classes (1 sample): 0
Large classes (>=2 samples): 295

=== Class Distribution ===
  运动服装: 2326 samples
  奢侈品箱包: 2053 samples
  皮具配饰,钱包: 1699 samples
  休闲服装: 1653 samples
  运动休闲裤: 1438 samples
  玩具模型: 1039 samples
  儿童玩具,轨道玩具: 1031 samples
  儿童玩具,教育玩具: 1014 samples
  玩具车辆: 980 samples
  儿童玩具,益智玩具: 935 samples
  ... and 285 more classes

=== Data Split ===
Small class samples: 0
Large class samples: 55376
Small class ratio: 0.000
Large class ratio: 1.000

=== Sampling Plan ===
From small classes: 0 samples
From large classes: 19148 samples
Total: 19148 samples

Sampling 19148 from large classes (stratified sampling)...
Large classes sampled: 19148 samples (stratifie

In [8]:
train_df_sample.head()

,filename,class,Usage,group,flag
13627,3728212.jpg,1859,Private,37,"运动鞋,洗涤用品"
11986,4867774.jpg,2231,Private,43,劳保鞋
17341,7227083.jpg,9054,Private,344,"家居用品,床上用品"
17753,3394203.jpg,8278,Public,251,"香水,香氛产品"
409,2562261.jpg,8695,Private,291,"化妆品,美妆用品"


### add additional train set

In [9]:
s3_prefix = "projects/nova-image-tagging/data/Product10K/test"

# Generate JSONL
with open(f'data/merged_data/{version}/nova_sft_trainset.jsonl', 'a', encoding='utf-8') as f:
    for idx, row in train_df_sample.iterrows():
        gt_label = '{"result":"' + row['flag'] + '"}'

        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            train_count += 1

print(f"Generated {train_count} records in nova_sft_dataset.jsonl")

Generated 20000 records in nova_sft_dataset.jsonl


### Add additional test set

In [10]:
s3_prefix = "projects/nova-image-tagging/data/Product10K/test"

# Generate JSONL
with open(f'data/merged_data/{version}/nova_sft_testset.jsonl', 'a', encoding='utf-8') as f:
    for idx, row in test_df_sample.iterrows():
        gt_label = '{"result":"' + row['flag'] + '"}'

        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            test_count += 1


print(f"Generated {test_count} records in nova_sft_dataset.jsonl")

Generated 1000 records in nova_sft_dataset.jsonl


In [11]:
# import json

# with open(f'data/merged_data/{version}/nova_sft_trainset.jsonl', 'r') as f:
#     for i, line in enumerate(f.readlines()):
#         if i > 1: 
#             break
#         if line.strip():
#             line = json.loads(line.strip())
#             print(json.dumps(line, indent=2, ensure_ascii=False))

### shuffle trainset

In [12]:
import random

# Set random seed
random.seed(42)

# Read all lines
with open(f'data/merged_data/{version}/nova_sft_trainset.jsonl', 'r') as f:
    lines = f.readlines()

print(f"{lines[:3]}")
# Shuffle the lines
random.shuffle(lines)
# print(f"lines: {len(lines)}")
# lines = lines[:19000]
# print(f"lines: {len(lines)}")
print(f"{lines[:3]}")

# Write back
with open(f'data/merged_data/{version}/nova_sft_trainset.jsonl', 'w') as f:
    f.writelines(lines)

['{"schemaVersion": "bedrock-conversation-2024", "system": [{"text": "##ROLE##\\nYou are an advanced image classification specialist analyzing e-commerce product images to identify multiple relevant category tags from the ##REFERENCE_CATEGORIES##. Your primary focus is comprehensive coverage (recall) over strict precision.\\n\\n##PRIMARY_TASK##\\nAnalyze the provided image and identify **UP TO 5 MOST RELEVANT** category tags from ##REFERENCE_CATEGORIES##. You are **STRONGLY ENCOURAGED** to output multiple tags when the image could reasonably belong to several categories or when there\'s reasonable uncertainty.\\n\\n##CRITICAL_MULTI_LABEL_REQUIREMENTS##\\n\\n**IMPORTANT: Multiple tags Are Expected and Preferred**\\n- **Output 1-3 tags** whenever multiple categories from ##REFERENCE_CATEGORIES## are plausible\\n- Think of this as providing comprehensive classification alternatives, not just the single best match\\n- Consider related categories, different levels of specificity, and variou

### get the statistics of nova_sft_trainset

In [ ]:
import json
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd

def obtain_stats(version, split):
    # Read the JSONL file and extract categories
    categories = []

    with open(f'data/merged_data/{version}/nova_sft_{split}set.jsonl', 'r') as f:
        for line in f:
            if line.strip():
                data = json.loads(line.strip())
                # Look for assistant messages
                for message in data.get('messages', []):
                    if message.get('role') == 'assistant':
                        content = message.get('content', [])
                        for item in content:
                            if 'text' in item:
                                try:
                                    # Parse the JSON string in the text field
                                    result_data = json.loads(item['text'])
                                    if 'result' in result_data:
                                        # Split by comma and add each category
                                        cats = result_data['result'].split(',')
                                        categories.extend([cat.strip() for cat in cats])
                                except json.JSONDecodeError:
                                    continue

    # Count categories
    category_counts = Counter(categories)

    # print(f"Total categories found: {len(categories)}")
    # print(f"Unique categories: {len(category_counts)}")
    # print(f"Top 20 categories:")
    # for cat, count in category_counts.most_common(20):
    #     print(f"  {cat}: {count}")

    # Create DataFrame for easier manipulation
    df_stats = pd.DataFrame(list(category_counts.items()), columns=['Category', 'Count'])
    df_stats = df_stats.sort_values('Count', ascending=False)
    
    # # Plot top 20 categories
    # plt.figure(figsize=(15, 8))
    # top_20 = df_stats.head(20)
    # plt.barh(range(len(top_20)), top_20['Count'])
    # plt.yticks(range(len(top_20)), top_20['Category'])
    # plt.xlabel('Count')
    # plt.title('Top 20 Categories in Nova SFT Training Set')
    # plt.gca().invert_yaxis()
    # plt.tight_layout()
    # plt.show()

    # Export results
    df_stats.to_csv(f'data/merged_data/{version}/{split}_stats.csv', index=False)
    print(f"Statistics exported to data/merged_data/{version}/{split}_stats.csv")

    return df_stats


In [15]:
train_stats = obtain_stats(version, 'train')
train_stats.head()

Total categories found: 30184
Unique categories: 405
Top 20 categories:
  儿童玩具: 1004
  运动服装: 774
  奢侈品箱包: 686
  运动鞋: 645
  钱包: 644
  皮具配饰: 568
  休闲服装: 557
  玩具模型: 529
  运动休闲裤: 484
  化妆品: 473
  休闲鞋: 388
  鞋类: 358
  教育玩具: 345
  轨道玩具: 344
  刀具: 336
  玩具车辆: 329
  益智玩具: 315
  服饰配件: 300
  打火器（无燃气）: 298
  保健品: 281
Statistics exported to data/merged_data/v1/train_stats.csv


,Category,Count
54,儿童玩具,1004
46,运动服装,774
2,奢侈品箱包,686
3,运动鞋,645
1,钱包,644


In [16]:
test_stats = obtain_stats(version, 'test')
test_stats.head()

Total categories found: 1504
Unique categories: 306
Top 20 categories:
  刀具: 80
  打火器（无燃气）: 55
  运动服装: 30
  充电宝: 29
  压罐喷雾: 27
  烟花爆竹: 27
  儿童玩具: 26
  电子打火器: 25
  奢侈品箱包: 24
  钱包: 22
  运动鞋: 20
  皮具配饰: 19
  儿童包: 15
  儿童自行车座椅: 15
  匕首: 15
  婴童帽子: 15
  射钉弹: 15
  火药: 15
  打火石、打火棒: 15
  指虎: 15
Statistics exported to data/merged_data/v1/test_stats.csv


,Category,Count
3,刀具,80
18,打火器（无燃气）,55
48,运动服装,30
8,充电宝,29
25,烟花爆竹,27
